# 第8章: 幻覚を避ける

- [レッスン](#lesson)
- [演習](#exercises)
- [サンプルプレイグラウンド](#example-playground)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を確立してください。

In [ ]:
# Pythonの組み込み正規表現ライブラリをインポート
import re
import boto3
import json

# utilsパッケージからhintsモジュールをインポート
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# IPythonストアからMODEL_NAME変数を取得
%store -r MODEL_NAME
%store -r AWS_REGION

client = boto3.client('bedrock-runtime',region_name=AWS_REGION)

def get_completion(prompt, system='', prefill=''):
    body = json.dumps(
        {
            "anthropic_version": '',
            "max_tokens": 2000,
            "messages":[
              {"role": "user", "content": prompt},
              {"role": "assistant", "content": prefill}
            ],
            "temperature": 0.0,
            "top_p": 1,
            "system": system
        }
    )
    response = client.invoke_model(body=body, modelId=MODEL_NAME)
    response_body = json.loads(response.get('body').read())

    return response_body.get('content')[0].get('text')

---

## レッスン

悪いニュースがあります: **Claudeは時々「幻覚」を起こし、真実ではない、または正当化されない主張をすることがあります**。良いニュースは、幻覚を最小限に抑えるテクニックがあるということです。
				
以下では、これらのテクニックのいくつかを紹介します。具体的には:
- Claudeに質問に対する答えを知らないと言う選択肢を与える
- 回答する前に証拠を見つけるようClaudeに求める

ただし、**幻覚を避けるには多くの方法があり**、このコースですでに学んだ多くのテクニックも含まれます。Claudeが幻覚を起こした場合は、複数のテクニックを試してClaudeの精度を向上させてください。

### 例

ここでは、一般的な事実に関する知識についての質問があり、**Claudeはできるだけ役立とうとするあまり、いくつかの大きなカバを幻覚で作り出しています**。

In [ ]:
# プロンプト
PROMPT = "史上最も重いカバは誰ですか？"

# Claudeの回答を出力
print(get_completion(PROMPT))

ここで試せる解決策は、「**Claudeに逃げ道を与える**」ことです — 回答を断ることができる、または確実に知っている場合にのみ回答するようClaudeに伝えます。

In [ ]:
# プロンプト
PROMPT = "史上最も重いカバは誰ですか？確実に答えを知っている場合にのみ回答してください。"

# Claudeの回答を出力
print(get_completion(PROMPT))

以下のプロンプトでは、ユーザーの質問にほぼ関連しているが完全には関連していない「ディストラクター情報」を含む長い文書をClaudeに与えています。**プロンプトの助けがなければ、Claudeはディストラクター情報に惑わされ**、2020年5月31日時点のMatterportの加入者数について不正確な「幻覚」の回答をしてしまいます。

**注意:** 次の章で学ぶように、**ベストプラクティスはテキストや文書の後に質問を置くこと**ですが、ここではプロンプトを読みやすくするために上部に置いています。プロンプトセルをダブルクリックして全文を確認してください（非常に長いです！）。

In [ ]:
# プロンプト
PROMPT = """<question>2020年5月31日の正確な日付におけるMatterportの加入者数は何人でしたか？</question>
以下の文書を読んでください。その後、<answer>タグ内に簡潔な数値回答を書いてください。

<document>
Matterport SEC filing 10-K 2023
Item 1. Business
Our Company
Matterportは、建築世界のデジタル化とデータ化をリードしています。
私たちは、建築世界のデジタル変革が、人々と建物や物理的空間との相互作用の方法を根本的に変えると信じています。
2011年の創業以来、Matterportの先駆的な技術は、建物、空間、場所のオンラインでのデジタル化、アクセス、管理の基準を設定してきました。私たちのプラットフォームの革新的なソフトウェア、空間データ駆動型データサイエンス、3Dキャプチャ技術は、世界最大の資産クラスである建物や物理的空間を長年オフラインで活用不足にしていた障壁を打ち破りました。私たちは、建築世界のデジタル化とデータ化が、大きな運用効率と不動産価値の向上をもたらし続けると信じており、Matterportはこの巨大なグローバル変革をリードするプラットフォームであると考えています。
世界は急速にオフラインからオンラインへ移行しています。
デジタル変革は、今日のあらゆるビジネスと産業に強力で持続的な影響を与えています。International Data Corporation（IDC）によると、2020年から2023年にかけてデジタル変革に6.8兆ドル以上の直接投資が行われ、グローバルデジタル変革支出は2026年に3.4兆ドルに達すると予測されており、5年間の複合年間成長率（CAGR）は16.3％、デジタルツイン投資は5年間のCAGRが35.2％と予想されています。この世俗的なシフトに伴い、建築世界を物理的からデジタルへ移行させる需要も高まっています。それにもかかわらず、建物や空間の大多数はオフラインのままで、デジタル化されていません。Savillsによると、2021年時点で全不動産価値は327兆ドルと推定されていますが、今日でもその大部分はオフラインのままであり、デジタル変革による浸透率は0.1％未満と推定されています。
Matterportは、建築世界のデジタル化の必要性の増大と、建物や空間の理解を促進する空間データ（建物や空間の根底にある固有の詳細）の力を最初に認識した企業の1つです。過去には、技術が物理的な道路地図を、今日私たちが頼りにしているデータ豊富なデジタル地図や位置情報サービスへと進化させました。Matterportは現在、建物をデジタル化し、データ豊富な環境を作り出すことで、私たちが捉える各空間の理解と可能性を大幅に向上させています。ボタン一つで瞬時に都市間の最速ルートを学んだり、最寄りのコーヒーショップを見つけたりできるように、Matterportの建物に関する空間データは、世界中の不動産や空間に関する豊富な洞察と学びを解き放ちます。さらに、今日の地理空間マッピングプラットフォームがマッピングデータを産業界に公開し、ライドシェアリング、電子商取引、フードデリバリーマーケットプレイス、さらには短期レンタルやホームシェアリングなどの新しいビジネスモデルを生み出したように、Matterportの構造化された空間データへのオープンアクセスは、ホスピタリティ、施設管理、保険、建設、不動産、小売などの分野で新たな機会とビジネスモデルを可能にしています。
私たちは、建築世界のデジタル化の総アドレス可能市場機会が2400億ドル以上であり、市場が成熟すると1兆ドルにも達する可能性があると考えています。
これは、世界に現在存在する40億以上の不動産と200億の空間という世界の建物ストックに関する私たちの分析、モデリング、理解に基づいています。人工知能（AI）、機械学習（ML）、ディープラーニング（DL）技術の助けを借りて、強力な空間データ駆動型の不動産洞察と分析からの追加の収益化機会により、建築世界のデジタル化とデータ化の総アドレス可能市場は1兆ドル以上に達すると信じています。
私たちの空間データプラットフォームとデジタルツインの捕捉は、多様な産業とユースケースにわたって価値を提供します。大規模小売業者は数千の店舗の場所をリモートで管理でき、不動産業者は数百の物件と何千人もの訪問者に同時にバーチャルオープンハウスを提供でき、不動産開発業者は建設プロセス全体をより詳細かつ迅速に監視でき、保険会社はより正確に請求と引受評価を文書化し、効率的かつ精密に評価できます。Matterportは、事実上あらゆるサイズ、形状、場所の不動産に関する重要なデジタル体験、ツール、情報を加入者に提供します。
約10年間にわたり、私たちは空間データプラットフォームを成長させ、その機能を拡張して、最も詳細で正確、データが豊富なデジタルツインを作成してきました。さらに、私たちの3D再構成プロセスは完全に自動化されており、世界中のあらゆるタイプ、形状、サイズの何百万もの建物や空間に同等の精度でスケールアップできるソリューションを提供しています。私たちのサービスの普遍的な適用可能性は、多様な垂直市場や地理的領域にわたってMatterportに重要なスケールとリーチを提供しています。2022年12月31日時点で、私たちの加入者ベースは前年同期の503,000人から約39％増加して701,000人以上となり、デジタルツインは170か国以上に到達しています。私たちは複数の産業にわたって280億平方フィート以上の空間をデジタル化しており、これは市場の残りの部分に対して重要なスケールと成長を示しています。

私たちは世界中の建物をデータに変換し続けるにつれて、空間データプラットフォームを拡張して、産業全体の加入者の不動産計画、開発、管理、インテリジェンスをさらに変革し、建築世界のデファクトの建物およびビジネスインテリジェンスエンジンになることを目指しています。私たちは、不動産、建築、エンジニアリング、建設（AEC）、小売、保険、政府を含む産業全体の企業、企業、機関にとって、空間データとそれに基づく洞察への需要が急速に成長し続けると信じています。
私たちは、デジタル化とデータ化がこの巨大なカテゴリーと資産クラス全体で成長のための大きな未開拓の機会を表していると信じています。設計と開発の初期段階からマーケティング、運営、保険、建物の修理とメンテナンスに至るまで、私たちのプラットフォームのソフトウェアと技術は、加入者にコスト削減、収益増加、建物や空間の最適な管理のための重要なツールと洞察を提供します。私たちは、数千億ドルの未実現の利用と運用効率が、私たちの空間データプラットフォームの力を通じて建築世界で解放できると信じています。私たちのプラットフォームとデータソリューションは産業と建物カテゴリー全体で普遍的に適用可能であり、Matterportに大きな利点を与えています。私たちはこの大きな市場機会全体に対応し、世界最大の資産クラスであると信じているものの価値を高めることができます。
加入者に価値を提供してきた実績があり、私たちの提供物にはソフトウェアサブスクリプション、データライセンシング、サービス、製品ハードウェアが含まれます。2022年12月31日時点で、私たちの加入者ベースにはFortune 1000企業の24％以上が含まれており、上位10社の加入者から生成される総収益は10％未満です。2025年までに、収益の80％以上がソフトウェアサブスクリプションとデータライセンスソリューションから来ると予想しています。私たちの革新的な3Dキャプチャ製品であるPro2カメラとPro3カメラは、3D建物および不動産視覚化エコシステムの形成に不可欠な役割を果たしてきました。Pro2カメラとPro3カメラは、私たちのソリューションの採用を促進し、独自の高品質でスケールされたデータセットを生成し、これによって私たちの独自のAIソフトウェアエンジンであるCortexがデジタルツイン作成のパイオニアエンジンになることを可能にしました。Pro2カメラによって最初に促進されたこのデータ優位性により、私たちはキャプチャデバイスに依存しないプラットフォームを開発し、産業や地理にまたがる加入者のために新しい建物と不動産の洞察を生成できるようになりました。
私たちは最近、急速な成長を経験しています。加入者は2018年12月31日から2022年12月31日までの期間に約49倍に成長しました。2022年12月31日に終了した年度の収益は、2021年12月31日に終了した年度の約1億1120万ドルから約22％増加して1億3610万ドルになりました。2022年12月31日に終了した年度の粗利益は、2021年12月31日に終了した年度の6000万ドルから810万ドルまたは14％減少して5180万ドルになりました。これは主に2022年度のグローバルサプライチェーンの制約による特定の混乱的および増分的コストに起因します。既存の加入者が生成するサブスクリプション収益を維持し成長させる能力は、私たちのビジネスの健全性と将来の成長見通しの重要な指標です。私たちは、comparable期間にわたる同じ顧客セットからのネットドル拡大率を測定することで、この分野でのパフォーマンスを追跡しています。2022年12月31日に終了した3か月間のネットドル拡大率103％は、私たちのプラットフォームの粘着性と成長可能性を示しています。
我々の産業と市場機会
今日、建物や空間の大多数はデジタル化されていません。私たちは、現在のサービス可能なアドレス可能市場には約13億の空間が含まれていると推定しており、主に不動産および旅行・ホスピタリティセクターからのものです。2022年12月31日時点で約920万の空間を管理しており、住宅および商業不動産、施設管理、小売、AEC、保険および修理、旅行およびホスピタリティを含む様々なエンドマーケットにわたって世界の建物ストックへの浸透と足跡の拡大を続けています。私たちは、総アドレス可能市場が40億以上の建物と200億の空間を世界的に含み、2400億ドル以上の市場機会をもたらすと推定しています。Matterportの独自の空間データライブラリと不動産データサービスが成長し続けるにつれて、この機会は建物ストックの規模と建物で利用可能な未開拓の価値創造に基づいて1兆ドル以上に増加する可能性があると信じています。COVID-19パンデミックによって作り出された制約は、私たちが過去10年間にわたって多様な産業と市場のために開発してきたスケールされた3Dキャプチャソリューションの重要性をさらに強化し加速させました。

我々の空間データプラットフォーム
概要
私たちの技術プラットフォームは、広範なデジタルキャプチャデバイスから収集された空間データを使用して、物理的な建物や空間を寸法的に正確で写真のようにリアルなデジタルツインに変換し、加入者が以前は利用できなかった建物情報と洞察にアクセスできるようにします。
この巨大な市場で約10年間にわたる最初の参入者として、私たちは業界をリードする3D再構成技術を開発およびスケールアップしてきました。これは、Cortexと呼ばれる私たちの独自のAI駆動ソフトウェアエンジンによって強化されており、機械学習を使用して建物構造全体の写真のようにリアルな3D仮想表現を再現します。これには、コンテンツ、設備、家具が含まれます。完成品は、物理的空間の詳細かつダイナミックな複製であり、Webブラウザからスマートフォンを含むあらゆるデバイスで探索、分析、カスタマイズできます。大規模な商業ビルでさえも管理する力が各加入者の手のひらにあり、これは私たちの高度な技術と空間データ技術スタック全体にわたる画期的なイノベーションによって可能になりました。
私たちの空間データプラットフォームの主要要素には以下が含まれます：
•オフラインの建物をオンラインに。従来、顧客は建物や空間を理解し評価するために現地訪問を行う必要がありました。写真や間取り図は役立つ場合がありますが、これらの二次元（2D）表現形式には限られた情報しかなく、静的で硬直的な傾向があり、そのため各建物や空間の全体的な理解に不可欠な対話的要素が欠けています。私たちの独自のAIソフトウェアであるCortexのAI駆動機能により、物理的オブジェクトの表現はもはや静的な2D画像に限定されず、物理的な訪問を排除できます。Cortexは建物や空間をオフラインからオンラインに移行させ、顧客がリアルタイムでどこからでもオンデマンドでアクセスできるようにします。加入者が建物をスキャンした後、私たちの視覚化アルゴリズムはスキャンを通じてキャプチャされた平坦な2D画像から空間位置と奥行きを正確に推論し、それらを高忠実度で精密なデジタルツインモデルに変換します。これにより、各デジタルツインがプロフェッショナルグレードの画質であることを保証する完全に自動化された画像処理パイプラインが作成されます。
•空間データによって駆動。私たちはデータ駆動型の企業です。空間の各増分キャプチャにより、私たちの空間データライブラリの豊かさと深さが増します。空間データは、人工環境の建物や空間の根底にある固有で特異な詳細を表します。Cortexは、長年にわたって蓄積してきた数十億のデータポイントの幅広さを使用して、デジタルツインの3D精度を向上させます。私たちは加入者がデジタルツイン内のオブジェクトの高さ、位置、その他の特性を正確に特定するのを支援します。私たちの洗練されたアルゴリズムは、データに基づく洞察を生成することで加入者に重要な商業的価値も提供し、不動産に関する評価や決定を自信を持って行うことができます。例えば、不動産開発業者は特定の窓から入る自然な熱と日光の量を評価でき、小売業者は各店舗のレイアウトが同じレベルのコードとブランド要件を満たしていることを確認でき、工場は機械のレイアウトが仕様と場所のガイドラインを満たしていることを確認できます。2022年12月31日時点で約920万の空間を管理しており、私たちの空間データライブラリは建築世界に関する情報のクリアリングハウスとなっています。
•AIとMLによって強化。人工知能と機械学習技術は、空間データを効果的に活用して、動的でリアルで対話的で情報豊富で複数の視点からの表示を可能にする堅牢な仮想体験を作成します。AIとMLはまた、毎日のスキャンに高価なカメラを不要にします—加入者は今やスマートフォンのボタンを押すだけで空間をスキャンできます。その結果、Matterportはデバイスに依存しないプラットフォームとなり、建築世界のデジタル化とインデックス化というミッションに向けてより迅速にスケールアップし推進することができます。
私たちの加入者への価値提案は、設計と建設から保守と運用、販促、販売、リース、保険、修理、修復、セキュリティ、金融に至るまで、デジタル建物ライフサイクル全体をカバーするように設計されています。その結果、私たちは加入者の不動産ポートフォリオ全体をオフラインからオンラインに移行させることで短期的および長期的な投資収益率を向上させる機会を発見するのを支援しながら、加入者とともに収益を成長させるユニークな立場にあると信じています。
ユビキタスキャプチャ
Matterportは3D空間キャプチャの標準となりました。私たちの技術プラットフォームは、世界中の加入者が建物や空間のインタラクティブで寸法的に正確なデジタルツインを迅速、簡単、正確にデジタル化、カスタマイズ、管理できるようにします。
Matterportプラットフォームは、LiDAR、球面、3D、360度カメラ、およびスマートフォンなど、幅広いキャプチャデバイスと連携するように設計されており、すべての加入者のキャプチャニーズに対応します。これにより、あらゆるサイズ、スケール、複雑さの空間を、いつでもどこでもキャプチャする柔軟性が提供されます。
•Matterport Pro3は、以前のバージョンよりも高速に不動産をスキャンし、プロジェクトの完了を加速させる最新の3Dカメラです。Pro3は屋内外の空間の最高精度スキャンを提供し、速度、忠実度、多用途性、精度のために設計されています。1回のスイープで20秒未満、100メートル先まで3Dデータをキャプチャし、Pro3の超高速、高精度LiDARセンサーは何時間も動作し、あらゆる条件で何百万もの測定を行います。
•Matterport Pro2は、世界中で何百万もの空間を高い忠実度、精度、速度、簡単さでキャプチャするために使用されてきた私たちの独自の3Dカメラです。500,000平方フィート以上のサイズの建物をキャプチャできる能力により、多くの住宅、商業、工業、大規模不動産のカメラの選択肢となっています。
•360度カメラ。Matterportは市場で入手可能な360度カメラの選択をサポートしています。これらの手頃な価格のポケットサイズのデバイスは、高忠実度の精密なキャプチャを提供し、小さな家、コンドミニアム、短期賃貸、アパートなどのキャプチャに適しています。これらのデバイスの球面レンズ画像キャプチャ技術は、Cortexにパノラマを業界をリードするデジタルツインに変換するための堅牢で詳細な画像データを提供します。
•LEICA BLK360。Leicaとのパートナーシップを通じて、私たちの3D再構成技術とAI駆動ソフトウェアエンジンであるCortexは、この強力なLiDARカメラをMatterportデジタルツインを作成するための超精密キャプチャデバイスに変換します。これは、正確な精度が必要なAECプロフェッショナルにとって選択されるソリューションです。
•スマートフォンキャプチャ。私たちのキャプチャアプリはiOSとAndroid用に市販されています。Matterportのスマートフォンキャプチャソリューションは3Dキャプチャを民主化し、2020年5月のMatterport for iPhoneの初期導入以来、最新のiPhoneデバイスで誰でも簡単にアクセスして建物や空間を3Dでデジタル化できるようになりました。2021年4月には、Android Captureアプリの正式リリースを発表し、Androidユーザーが迅速かつ簡単に建物や空間を没入型3Dでキャプチャできるようになりました。2022年2月には、Matterport Axisを発売しました。これはスマートフォンを保持するモーター駆動マウントで、Matterport Captureアプリと一緒に使用して、速度、精度、一貫性を向上させながら、あらゆる物理的空間の3Dデジタルツインをキャプチャできます。
Cortexと3D再構成（Matterportデジタルツイン）
2022年12月31日時点で、約920万の空間を管理し、約280億平方フィートの空間をキャプチャした空間データライブラリを持ち、私たちは高度なMLとDL技術を使用して、キャプチャした空間データをアルゴリズム的に任意の物理的空間の正確な3Dデジタル再現に変換します。この知的で自動化された3D再構成は、Cortexによって可能になります。Cortexは私たちのAI駆動ソフトウェアエンジンで、ディープラーニングニューラルネットワークを含み、空間データライブラリを使用して建物や空間がどのように階と部屋に分割されているか、ドアや開口部がどこにあるか、どのタイプの部屋が存在するかを理解し、それらの形式を寸法的な正確さでダイナミックで写真のようにリアルなデジタルツインにコンパイルおよび整列します。Cortexの他のコンポーネントには、建物や空間内のコンテンツを識別し分類するためのAI駆動コンピュータビジョン技術や、家具や設備からドア、窓、照明器具、消火スプリンクラー、非常階段まですべてを識別しセグメント化するオブジェクト認識技術が含まれます。私たちの高度にスケーラブルな人工知能プラットフォームにより、加入者はボタンをクリックするだけで強力な拡張された建物データと洞察にアクセスできます。

Matterportデジタルツインの背後にある科学：Cortex AIのハイライト
MatterportはCortexで動作
Cortexは、任意の建物や空間のデジタルツインを作成するための精密なディープラーニングニューラルネットワークを含むAI駆動ソフトウェアエンジンです。Pro2カメラとPro3カメラでキャプチャした独自の空間データを使用して開発されたCortexは、高度な精度と正確性を提供しながら、日常のデバイスを使用した3Dキャプチャを可能にします。
一般的なニューラルネットワークは現実世界の3D再構成に苦戦します。Matterport最適化ネットワークは、より正確で堅牢な結果を提供します。単なる生のトレーニングデータ以上に、Matterportのデータセットにより、新しいニューラルネットワークアーキテクチャを開発し、何百万もの状況でユーザーの行動と実世界のデータに対して評価することができます。
•ディープラーニング：各空間の詳細なニューラルネットワークデータアーキテクチャを接続し最適化することは、堅牢で高精度の3Dデジタルツインを作成する鍵です。Cortexは、何百万もの建物や空間から集約されたMatterportの豊富な空間データと、世界中の何万人もの加入者によって提供されたそれらのデータの人間による注釈に対して、各3Dモデルを評価し最適化します。Cortexの評価能力と3D再構成のデータ駆動型最適化により、幅広い建物構成、空間、環境にわたって一貫した高精度の結果が得られます。
•ダイナミック3D再構成：2Dビジュアルと静的画像からスケールで精密な3D空間データを作成するには、複数の視点からの写真のようにリアルで詳細なデータと、Cortexのニューラルネットワークと学習能力を訓練および最適化する何百万もの空間の組み合わせが必要です。これにより、任意の空間の3D再構成が向上します。Cortexの機能と私たちの3Dキャプチャ技術におけるリアルタイムの空間アラインメントアルゴリズムを組み合わせることで、進行中の作業の直感的な「プレビュー」が作成され、加入者はコンテンツをインタラクティブにリアルタイムで操作できます。
•コンピュータビジョン：Cortexは、デジタルツインの価値を高める強力な機能のスイートを可能にします。これには、部屋や部屋内のオブジェクトの自動測定、3Dからの自動2D高解像度フォトギャラリー作成、プライバシー保護のための自動顔のぼかし、カスタムビデオ、ウォークスルー、自動部屋ラベリング、オブジェクト認識が含まれます。
•高度な画像処理：Matterportの計算写真アルゴリズムは、各デジタルツインがプロフェッショナルグレードの画質であることを保証するために、完全に自動化された画像処理パイプラインを作成します。私たちの特許取得技術により、3Dキャプチャは単一のボタンを押すだけで簡単になります。Matterportのソフトウェアと技術が残りのステップを管理し、ホワイトバランスとカメラ固有の色補正、高ダイナミックレンジのトーンマッピング、ノイズ除去、ヘイズ除去、シャープニング、彩度、その他の画質を向上させる調整を行います。
空間データとAI駆動の洞察
すべてのMatterportデジタルツインには、建物、部屋、物理的空間に関する広範な情報が含まれています。データはAI駆動のCortexエンジンを使用します。Matterportデジタルツイン自体に加えて、私たちの空間データは、精密な建物の幾何学と構造的詳細、建物の内容、設備、状態、さらに空間内の多くの視点からの高解像度画像と写真のようにリアルな詳細で構成されています。Cortexは、私たちが深層空間インデックス作成と呼ぶ技術を採用しています。深層空間インデックス作成は、人工知能、コンピュータビジョン、ディープラーニングを使用して、各空間、その構造、およびその内容に関する重要な詳細を精密かつ忠実に識別し伝達します。私たちは、Matterport加入者が任意の建物の相互運用可能なデジタルシステムを活用できる堅牢な空間データ標準を作成しました。
デジタルツインの構築を通じて加入者に高度にインタラクティブなデジタル体験を提供することに加えて、私たちは各加入者に対して2つの質問をします：（1）彼らの建物や物理的空間について重要なことは何か、（2）この空間についてどのような学びと洞察を提供できるか？私たちのAI駆動Cortexエンジンは、空間データライブラリを使用してこれらの質問に答え、集約された不動産トレンドと運用および評価の洞察を提供するのに役立ちます。さらに、Matterportプラットフォームエコシステムが拡大し続けるにつれて、私たちの加入者、パートナー、その他のサードパーティ開発者は、豊富な空間データレイヤーから収穫できる洞察の幅と深さをさらに広げるために独自のツールを持ち込むことができます。
拡張可能なプラットフォームエコシステム
Matterportは、2022年12月31日時点で約920万の空間を管理し、約280億平方フィートをキャプチャした、世界最大かつ最も正確な空間データライブラリを提供しています。私たちの空間データプラットフォームの多用途性と広範なエンタープライズソフトウェア開発キットおよびアプリケーションプログラミングインターフェース（API）により、地理と垂直市場によってMatterportの価値提案を拡張するチャネルとパートナーの堅牢なグローバルエコシステムを開発することができました。私たちは、ターゲット市場全体で統合されたMatterportソリューションを促進するために、パートナーとその加入者との広範なワークフロー統合を継続的に展開する予定です。また、簡単に展開およびアクセスできるMatterportソフトウェアアドオンで空間データプラットフォームの力を拡張するサードパーティソフトウェアマーケットプレイスも開発しています。このマーケットプレイスにより、開発者は新しいアプリケーションと空間データマイニングツールを構築し、Matterport 3Dエクスペリエンスを強化し、私たちのコア製品を補完する新しい生産性と不動産管理ツールを作成できます。サードパーティ開発者によって作成されたこれらの付加価値機能は、スケーラブルな新しい収益ストリームを可能にし、オンラインマーケットプレイスを通じて加入者に展開される各アドオンのサブスクリプションとサービス収益をMatterportが共有します。私たちのプラットフォームエコシステムのネットワーク効果は、ビジネスの成長に貢献し、加入者の粘着性とユーザーエンゲージメントを高めることで、将来の成長をさらに後押しすると信じています。
Matterportのアドオンと拡張機能の例には以下が含まれます：
•アドオン：Encircle（より迅速な請求処理のための使いやすいフィールドドキュメンテーションツール）、WP Matterport Shortcode（Matterportショートコードを使用してMatterportを迅速かつ簡単に埋め込むことができる無料のWordpressプラグイン）、WP3D Models（WordPress + Matterport統合プラグイン）、Rela（リスティング用のオールインワンマーケティングソリューション）、CAPTUR3D（Matterportデジタルツインに価値を拡張するオールインワンコンテンツ管理システム）、Private Model Embed（企業が追加のユーザーライセンスを必要とせずに、企業ネットワーク上の大規模な従業員グループとデジタルツインを非公開で共有できる機能）、Views（グループや大規模組織が異なるタスクを管理するための別個の権限ベースのワークフローを作成できる新しいワークグループコラボレーションフレームワーク）、Guided Tours and Tags（訪問者の関心に合わせて商業または住宅空間の誘導型バーチャルツアーを作成することで訪問者体験を向上させるツール）。Amazon Web Services（AWS）IoT TwinMakerとのプライベートベータ統合を公開し、エンタープライズ顧客がIoTデータを視覚的に没入型で寸法的に正確なMatterportデジタルツインにシームレスに接続できるようにしました。
•サービス：Matterport ADA Compliant Digital Twin（アメリカ障害者法に準拠したデジタルツインを提供するソリューション）およびEnterprise Cloud Software Platform（あらゆるサイズや形状の建物や空間のデジタルツインを作成、公開、管理する、エンタープライズ向けに再考されたクラウドソフトウェアプラットフォーム、屋内外を問わず）。
我々の競争力
私たちは、市場リーダーシップの成長を可能にする多くの競争力を持っていると信じています。私たちの競争力には以下が含まれます：
•Matterportプラットフォームの幅広さと深さ。私たちの中核的な強みは、カスタマイズなしでキャプチャから処理、産業まで多様な垂直市場と地理にわたる広範な到達範囲を持つオールインワンの空間データプラットフォームです。様々なエンタープライズシステムとシームレスに統合する能力により、私たちのプラットフォームは不動産、AEC、旅行とホスピタリティ、修理と保険、産業と施設を含む多様なエンドマーケットの不動産ライフサイクル全体にわたって価値を提供します。2022年12月31日時点で、私たちのグローバルリーチはFortune 1000企業の24％以上を含む170か国以上の加入者に及んでいます。
•市場リーダーシップと先行者優位性。Matterportは約10年前に建築世界のデジタル化とデータ化のカテゴリーを定義し、このカテゴリーでグローバルリーダーになりました。2022年12月31日時点で、私たちのプラットフォーム上に701,000以上の加入者がおり、約920万の空間を管理しています。私たちのリーダーシップは、主にデジタルツイン作成の先行者であったという事実に起因します。先行者優位性の結果として、私たちは継続的に複利効果を生み出し、リーダーシップの地位を強化する深くて豊かな空間データライブラリを蓄積しました。
•重要なネットワーク効果。プラットフォームに新しいキャプチャとデータが追加されるたびに、私たちのデータセットの豊かさと管理下の空間からの洞察の深さが増します。さらに、データを洞察に変換する能力と加入者による新しいデータキャプチャからの増分データを組み合わせることで、Matterportはプラットフォームの加入者向けの機能を開発することができます。私たちは建築世界の空間データライブラリを構築する先行者であり、データの収集と洞察の導出におけるリーダーシップは継続的に複利効果を生み出し、それらの洞察の関連性がさらに新しい加入者を引き付けます。
•加入者に価値ある不動産洞察を提供する生の素材としての巨大な空間データライブラリ。私たちの空間データライブラリの規模は、加入者に洞察を導出する上で重要な利点です。私たちの空間データライブラリは、Cortexにとって重要な基準点として機能し、Matterportが低コストのデジタルカメラやスマートフォンを含む幅広いカメラ技術を使用して強力な3Dデジタルツインを作成することを可能にします。2022年12月31日時点で、私たちのデータは約920万の管理下の空間と約280億平方フィートのキャプチャから得られています。その結果、私たちは不動産の洞察と分析を新しいレベルに引き上げ、様々な産業の加入者に利益をもたらしています。例えば、施設管理者は建物のレイアウトを作成するのに必要な時間を大幅に削減し、現場調査と建設済みモデリングのコストを大幅に削減することができます。AEC加入者は、各建設済み空間の分析を使用してドキュメンテーションを効率化し、容易に協力することができます。
•グローバルリーチとスケール。私たちは、AI駆動の空間データプラットフォームを世界中で継続的に拡大することに焦点を当てています。北米、ヨーロッパ、アジアに大きなプレゼンスを持ち、各地域にリーダーシップチームとゴートゥマーケットインフラストラクチャを有しています。ロンドン、シンガポール、そしてアメリカ合衆国全土にオフィスを構え、国際的な拡大を加速しています。2022年12月31日時点で、170か国以上に701,000以上の加入者がいます。私たちの空間データプラットフォームの地理に依存しない性質は、国際的に成長を続ける上で重要な利点であると信じています。
•10年間のR&Dとイノベーションを支える幅広い特許ポートフォリオ。2022年12月31日時点で、54件の発行済み特許と37件の特許出願中があります。私たちの成功は、約10年間のイノベーションへの集中に基づいています。イノベーションはMatterportの中心にあり、私たちは市場をリードする地位をさらに強化するためにR&Dへの投資を継続的に優先していきます。
•優れたキャプチャ技術。Matterportのキャプチャ技術プラットフォームは、建物や空間のMatterportデジタルツインを作成するために必要な幅広いキャプチャデバイスをサポートするソフトウェアフレームワークです。
これには、LiDARカメラ、360度カメラ、スマートフォン、Matterport Axis、Matterport Pro2およびPro3カメラのサポートが含まれます。Pro2カメラは私たちの空間データ優位性の基礎となり、Matterport対応のサードパーティキャプチャデバイスの配列でその優位性を拡大しました。2022年8月、業界をリードするデジタルツインクラウドプラットフォームの主要な更新とともに、Pro3カメラの出荷を開始しました。Matterport Pro3カメラは高度な3Dキャプチャデバイスで、起動時間の短縮、交換可能なバッテリー、軽量設計を特徴としています。Pro3カメラは屋内外で使用でき、速度、忠実度、多用途性、精度のために設計されています。Pro2カメラとともに、Pro3カメラの今後の販売が私たちのソリューションのさらなる採用を促進すると期待しています。Matterportは3Dキャプチャ体験を民主化し、あらゆる建物タイプと不動産ライフサイクルにおける加入者のニーズに対して、高忠実度で高精度の3Dデジタルツインを容易に利用できるようにしています。他の3Dキャプチャソリューションプロバイダーもありますが、真に寸法的に正確な3D結果を生成できるものはごくわずかで、さらに少数しか自動的に写真のようにリアルな3Dの最終製品をグローバルスケールで作成できません。この広範なキャプチャ技術の提供は、スマートフォンや360度カメラでキャプチャした写真から正確なデジタルツインを自動生成するAI駆動のCortexエンジンを訓練するための豊富な空間データライブラリなしでは不可能でしょう。
</document>"""

# Claudeの回答を出力
print(get_completion(PROMPT))

これをどのように修正すればよいでしょうか？長い文書に対する幻覚を減らす優れた方法は、**まずClaudeに証拠を集めさせることです**。

この場合、**まず関連する引用を抽出し、その引用に基づいて回答するようClaudeに指示します**。ここでそうすることで、Claudeは引用が質問に答えていないことを正しく認識します。

In [ ]:
# プロンプト
PROMPT = """<question>2020年5月31日の正確な日付におけるMatterportの加入者数は何人でしたか？</question>
以下の文書を読んでください。次に、<scratchpad>タグ内で、文書から最も関連性の高い引用を抽出し、それがユーザーの質問に答えているか、または十分な詳細が不足しているかを検討してください。その後、<answer>タグ内に簡潔な数値回答を書いてください。

<document>
Matterport SEC filing 10-K 2023
Item 1. Business
Our Company
Matterportは、建築世界のデジタル化とデータ化をリードしています。
私たちは、建築世界のデジタル変革が、人々と建物や物理的空間との相互作用の方法を根本的に変えると信じています。
2011年の創業以来、Matterportの先駆的な技術は、建物、空間、場所のオンラインでのデジタル化、アクセス、管理の基準を設定してきました。私たちのプラットフォームの革新的なソフトウェア、空間データ駆動型データサイエンス、3Dキャプチャ技術は、世界最大の資産クラスである建物や物理的空間を長年オフラインで活用不足にしていた障壁を打ち破りました。私たちは、建築世界のデジタル化とデータ化が、大きな運用効率と不動産価値の向上をもたらし続けると信じており、Matterportはこの巨大なグローバル変革をリードするプラットフォームであると考えています。
世界は急速にオフラインからオンラインへ移行しています。
デジタル変革は、今日のあらゆるビジネスと産業に強力で持続的な影響を与えています。International Data Corporation（IDC）によると、2020年から2023年にかけてデジタル変革に6.8兆ドル以上の直接投資が行われ、グローバルデジタル変革支出は2026年に3.4兆ドルに達すると予測されており、5年間の複合年間成長率（CAGR）は16.3％、デジタルツイン投資は5年間のCAGRが35.2％と予想されています。この世俗的なシフトに伴い、建築世界を物理的からデジタルへ移行させる需要も高まっています。それにもかかわらず、建物や空間の大多数はオフラインのままで、デジタル化されていません。Savillsによると、2021年時点で全不動産価値は327兆ドルと推定されていますが、今日でもその大部分はオフラインのままであり、デジタル変革による浸透率は0.1％未満と推定されています。
Matterportは、建築世界のデジタル化の必要性の増大と、建物や空間の理解を促進する空間データ（建物や空間の根底にある固有の詳細）の力を最初に認識した企業の1つです。過去には、技術が物理的な道路地図を、今日私たちが頼りにしているデータ豊富なデジタル地図や位置情報サービスへと進化させました。Matterportは現在、建物をデジタル化し、データ豊富な環境を作り出すことで、私たちが捉える各空間の理解と可能性を大幅に向上させています。ボタン一つで瞬時に都市間の最速ルートを学んだり、最寄りのコーヒーショップを見つけたりできるように、Matterportの建物に関する空間データは、世界中の不動産や空間に関する豊富な洞察と学びを解き放ちます。さらに、今日の地理空間マッピングプラットフォームがマッピングデータを産業界に公開し、ライドシェアリング、電子商取引、フードデリバリーマーケットプレイス、さらには短期レンタルやホームシェアリングなどの新しいビジネスモデルを生み出したように、Matterportの構造化された空間データへのオープンアクセスは、ホスピタリティ、施設管理、保険、建設、不動産、小売などの分野で新たな機会とビジネスモデルを可能にしています。
私たちは、建築世界のデジタル化の総アドレス可能市場機会が2400億ドル以上であり、市場が成熟すると1兆ドルにも達する可能性があると考えています。
これは、世界に現在存在する40億以上の不動産と200億の空間という世界の建物ストックに関する私たちの分析、モデリング、理解に基づいています。人工知能（AI）、機械学習（ML）、ディープラーニング（DL）技術の助けを借りて、強力な空間データ駆動型の不動産洞察と分析からの追加の収益化機会により、建築世界のデジタル化とデータ化の総アドレス可能市場は1兆ドル以上に達すると信じています。
私たちの空間データプラットフォームとデジタルツインの捕捉は、多様な産業とユースケースにわたって価値を提供します。大規模小売業者は数千の店舗の場所をリモートで管理でき、不動産業者は数百の物件と何千人もの訪問者に同時にバーチャルオープンハウスを提供でき、不動産開発業者は建設プロセス全体をより詳細かつ迅速に監視でき、保険会社はより正確に請求と引受評価を文書化し、効率的かつ精密に評価できます。Matterportは、事実上あらゆるサイズ、形状、場所の不動産に関する重要なデジタル体験、ツール、情報を加入者に提供します。
約10年間にわたり、私たちは空間データプラットフォームを成長させ、その機能を拡張して、最も詳細で正確、データが豊富なデジタルツインを作成してきました。さらに、私たちの3D再構成プロセスは完全に自動化されており、世界中のあらゆるタイプ、形状、サイズの何百万もの建物や空間に同等の精度でスケールアップできるソリューションを提供しています。私たちのサービスの普遍的な適用可能性は、多様な垂直市場や地理的領域にわたってMatterportに重要なスケールとリーチを提供しています。2022年12月31日時点で、私たちの加入者ベースは前年同期の503,000人から約39％増加して701,000人以上となり、デジタルツインは170か国以上に到達しています。私たちは複数の産業にわたって280億平方フィート以上の空間をデジタル化しており、これは市場の残りの部分に対して重要なスケールと成長を示しています。

私たちは世界中の建物をデータに変換し続けるにつれて、空間データプラットフォームを拡張して、産業全体の加入者の不動産計画、開発、管理、インテリジェンスをさらに変革し、建築世界のデファクトの建物およびビジネスインテリジェンスエンジンになることを目指しています。私たちは、不動産、建築、エンジニアリング、建設（AEC）、小売、保険、政府を含む産業全体の企業、企業、機関にとって、空間データとそれに基づく洞察への需要が急速に成長し続けると信じています。
私たちは、デジタル化とデータ化がこの巨大なカテゴリーと資産クラス全体で成長のための大きな未開拓の機会を表していると信じています。設計と開発の初期段階からマーケティング、運営、保険、建物の修理とメンテナンスに至るまで、私たちのプラットフォームのソフトウェアと技術は、加入者にコスト削減、収益増加、建物や空間の最適な管理のための重要なツールと洞察を提供します。私たちは、数千億ドルの未実現の利用と運用効率が、私たちの空間データプラットフォームの力を通じて建築世界で解放できると信じています。私たちのプラットフォームとデータソリューションは産業と建物カテゴリー全体で普遍的に適用可能であり、Matterportに大きな利点を与えています。私たちはこの大きな市場機会全体に対応し、世界最大の資産クラスであると信じているものの価値を高めることができます。
加入者に価値を提供してきた実績があり、私たちの提供物にはソフトウェアサブスクリプション、データライセンシング、サービス、製品ハードウェアが含まれます。2022年12月31日時点で、私たちの加入者ベースにはFortune 1000企業の24％以上が含まれており、上位10社の加入者から生成される総収益は10％未満です。2025年までに、収益の80％以上がソフトウェアサブスクリプションとデータライセンスソリューションから来ると予想しています。私たちの革新的な3Dキャプチャ製品であるPro2カメラとPro3カメラは、3D建物および不動産視覚化エコシステムの形成に不可欠な役割を果たしてきました。Pro2カメラとPro3カメラは、私たちのソリューションの採用を促進し、独自の高品質でスケールされたデータセットを生成し、これによって私たちの独自のAIソフトウェアエンジンであるCortexがデジタルツイン作成のパイオニアエンジンになることを可能にしました。Pro2カメラによって最初に促進されたこのデータ優位性により、私たちはキャプチャデバイスに依存しないプラットフォームを開発し、産業や地理にまたがる加入者のために新しい建物と不動産の洞察を生成できるようになりました。
私たちは最近、急速な成長を経験しています。加入者は2018年12月31日から2022年12月31日までの期間に約49倍に成長しました。2022年12月31日に終了した年度の収益は、2021年12月31日に終了した年度の約1億1120万ドルから約22％増加して1億3610万ドルになりました。2022年12月31日に終了した年度の粗利益は、2021年12月31日に終了した年度の6000万ドルから810万ドルまたは14％減少して5180万ドルになりました。これは主に2022年度のグローバルサプライチェーンの制約による特定の混乱的および増分的コストに起因します。既存の加入者が生成するサブスクリプション収益を維持し成長させる能力は、私たちのビジネスの健全性と将来の成長見通しの重要な指標です。私たちは、comparable期間にわたる同じ顧客セットからのネットドル拡大率を測定することで、この分野でのパフォーマンスを追跡しています。2022年12月31日に終了した3か月間のネットドル拡大率103％は、私たちのプラットフォームの粘着性と成長可能性を示しています。
我々の産業と市場機会
今日、建物や空間の大多数はデジタル化されていません。私たちは、現在のサービス可能なアドレス可能市場には約13億の空間が含まれていると推定しており、主に不動産および旅行・ホスピタリティセクターからのものです。2022年12月31日時点で約920万の空間を管理しており、住宅および商業不動産、施設管理、小売、AEC、保険および修理、旅行およびホスピタリティを含む様々なエンドマーケットにわたって世界の建物ストックへの浸透と足跡の拡大を続けています。私たちは、総アドレス可能市場が40億以上の建物と200億の空間を世界的に含み、2400億ドル以上の市場機会をもたらすと推定しています。Matterportの独自の空間データライブラリと不動産データサービスが成長し続けるにつれて、この機会は建物ストックの規模と建物で利用可能な未開拓の価値創造に基づいて1兆ドル以上に増加する可能性があると信じています。COVID-19パンデミックによって作り出された制約は、私たちが過去10年間にわたって多様な産業と市場のために開発してきたスケールされた3Dキャプチャソリューションの重要性をさらに強化し加速させました。

我々の空間データプラットフォーム
概要
私たちの技術プラットフォームは、広範なデジタルキャプチャデバイスから収集された空間データを使用して、物理的な建物や空間を寸法的に正確で写真のようにリアルなデジタルツインに変換し、加入者が以前は利用できなかった建物情報と洞察にアクセスできるようにします。
この巨大な市場で約10年間にわたる最初の参入者として、私たちは業界をリードする3D再構成技術を開発およびスケールアップしてきました。これは、Cortexと呼ばれる私たちの独自のAI駆動ソフトウェアエンジンによって強化されており、機械学習を使用して建物構造全体の写真のようにリアルな3D仮想表現を再現します。これには、コンテンツ、設備、家具が含まれます。完成品は、物理的空間の詳細かつダイナミックな複製であり、Webブラウザからスマートフォンを含むあらゆるデバイスで探索、分析、カスタマイズできます。大規模な商業ビルでさえも管理する力が各加入者の手のひらにあり、これは私たちの高度な技術と空間データ技術スタック全体にわたる画期的なイノベーションによって可能になりました。
私たちの空間データプラットフォームの主要要素には以下が含まれます：
•オフラインの建物をオンラインに。従来、顧客は建物や空間を理解し評価するために現地訪問を行う必要がありました。写真や間取り図は役立つ場合がありますが、これらの二次元（2D）表現形式には限られた情報しかなく、静的で硬直的な傾向があり、そのため各建物や空間の全体的な理解に不可欠な対話的要素が欠けています。私たちの独自のAIソフトウェアであるCortexのAI駆動機能により、物理的オブジェクトの表現はもはや静的な2D画像に限定されず、物理的な訪問を排除できます。Cortexは建物や空間をオフラインからオンラインに移行させ、顧客がリアルタイムでどこからでもオンデマンドでアクセスできるようにします。加入者が建物をスキャンした後、私たちの視覚化アルゴリズムはスキャンを通じてキャプチャされた平坦な2D画像から空間位置と奥行きを正確に推論し、それらを高忠実度で精密なデジタルツインモデルに変換します。これにより、各デジタルツインがプロフェッショナルグレードの画質であることを保証する完全に自動化された画像処理パイプラインが作成されます。
•空間データによって駆動。私たちはデータ駆動型の企業です。空間の各増分キャプチャにより、私たちの空間データライブラリの豊かさと深さが増します。空間データは、人工環境の建物や空間の根底にある固有で特異な詳細を表します。Cortexは、長年にわたって蓄積してきた数十億のデータポイントの幅広さを使用して、デジタルツインの3D精度を向上させます。私たちは加入者がデジタルツイン内のオブジェクトの高さ、位置、その他の特性を正確に特定するのを支援します。私たちの洗練されたアルゴリズムは、データに基づく洞察を生成することで加入者に重要な商業的価値も提供し、不動産に関する評価や決定を自信を持って行うことができます。例えば、不動産開発業者は特定の窓から入る自然な熱と日光の量を評価でき、小売業者は各店舗のレイアウトが同じレベルのコードとブランド要件を満たしていることを確認でき、工場は機械のレイアウトが仕様と場所のガイドラインを満たしていることを確認できます。2022年12月31日時点で約920万の空間を管理しており、私たちの空間データライブラリは建築世界に関する情報のクリアリングハウスとなっています。
•AIとMLによって強化。人工知能と機械学習技術は、空間データを効果的に活用して、動的でリアルで対話的で情報豊富で複数の視点からの表示を可能にする堅牢な仮想体験を作成します。AIとMLはまた、毎日のスキャンに高価なカメラを不要にします—加入者は今やスマートフォンのボタンを押すだけで空間をスキャンできます。その結果、Matterportはデバイスに依存しないプラットフォームとなり、建築世界のデジタル化とインデックス化というミッションに向けてより迅速にスケールアップし推進することができます。
私たちの加入者への価値提案は、設計と建設から保守と運用、販促、販売、リース、保険、修理、修復、セキュリティ、金融に至るまで、デジタル建物ライフサイクル全体をカバーするように設計されています。その結果、私たちは加入者の不動産ポートフォリオ全体をオフラインからオンラインに移行させることで短期的および長期的な投資収益率を向上させる機会を発見するのを支援しながら、加入者とともに収益を成長させるユニークな立場にあると信じています。
ユビキタスキャプチャ
Matterportは3D空間キャプチャの標準となりました。私たちの技術プラットフォームは、世界中の加入者が建物や空間のインタラクティブで寸法的に正確なデジタルツインを迅速、簡単、正確にデジタル化、カスタマイズ、管理できるようにします。
Matterportプラットフォームは、LiDAR、球面、3D、360度カメラ、およびスマートフォンなど、幅広いキャプチャデバイスと連携するように設計されており、すべての加入者のキャプチャニーズに対応します。これにより、あらゆるサイズ、スケール、複雑さの空間を、いつでもどこでもキャプチャする柔軟性が提供されます。
•Matterport Pro3は、以前のバージョンよりも高速に不動産をスキャンし、プロジェクトの完了を加速させる最新の3Dカメラです。Pro3は屋内外の空間の最高精度スキャンを提供し、速度、忠実度、多用途性、精度のために設計されています。1回のスイープで20秒未満、100メートル先まで3Dデータをキャプチャし、Pro3の超高速、高精度LiDARセンサーは何時間も動作し、あらゆる条件で何百万もの測定を行います。
•Matterport Pro2は、世界中で何百万もの空間を高い忠実度、精度、速度、簡単さでキャプチャするために使用されてきた私たちの独自の3Dカメラです。500,000平方フィート以上のサイズの建物をキャプチャできる能力により、多くの住宅、商業、工業、大規模不動産のカメラの選択肢となっています。
•360度カメラ。Matterportは市場で入手可能な360度カメラの選択をサポートしています。これらの手頃な価格のポケットサイズのデバイスは、高忠実度の精密なキャプチャを提供し、小さな家、コンドミニアム、短期賃貸、アパートなどのキャプチャに適しています。これらのデバイスの球面レンズ画像キャプチャ技術は、Cortexにパノラマを業界をリードするデジタルツインに変換するための堅牢で詳細な画像データを提供します。
•LEICA BLK360。Leicaとのパートナーシップを通じて、私たちの3D再構成技術とAI駆動ソフトウェアエンジンであるCortexは、この強力なLiDARカメラをMatterportデジタルツインを作成するための超精密キャプチャデバイスに変換します。これは、正確な精度が必要なAECプロフェッショナルにとって選択されるソリューションです。
•スマートフォンキャプチャ。私たちのキャプチャアプリはiOSとAndroid用に市販されています。Matterportのスマートフォンキャプチャソリューションは3Dキャプチャを民主化し、2020年5月のMatterport for iPhoneの初期導入以来、最新のiPhoneデバイスで誰でも簡単にアクセスして建物や空間を3Dでデジタル化できるようになりました。2021年4月には、Android Captureアプリの正式リリースを発表し、Androidユーザーが迅速かつ簡単に建物や空間を没入型3Dでキャプチャできるようになりました。2022年2月には、Matterport Axisを発売しました。これはスマートフォンを保持するモーター駆動マウントで、Matterport Captureアプリと一緒に使用して、速度、精度、一貫性を向上させながら、あらゆる物理的空間の3Dデジタルツインをキャプチャできます。
Cortexと3D再構成（Matterportデジタルツイン）
2022年12月31日時点で、約920万の空間を管理し、約280億平方フィートの空間をキャプチャした空間データライブラリを持ち、私たちは高度なMLとDL技術を使用して、キャプチャした空間データをアルゴリズム的に任意の物理的空間の正確な3Dデジタル再現に変換します。この知的で自動化された3D再構成は、Cortexによって可能になります。Cortexは私たちのAI駆動ソフトウェアエンジンで、ディープラーニングニューラルネットワークを含み、空間データライブラリを使用して建物や空間がどのように階と部屋に分割されているか、ドアや開口部がどこにあるか、どのタイプの部屋が存在するかを理解し、それらの形式を寸法的な正確さでダイナミックで写真のようにリアルなデジタルツインにコンパイルおよび整列します。Cortexの他のコンポーネントには、建物や空間内のコンテンツを識別し分類するためのAI駆動コンピュータビジョン技術や、家具や設備からドア、窓、照明器具、消火スプリンクラー、非常階段まですべてを識別しセグメント化するオブジェクト認識技術が含まれます。私たちの高度にスケーラブルな人工知能プラットフォームにより、加入者はボタンをクリックするだけで強力な拡張された建物データと洞察にアクセスできます。

Matterportデジタルツインの背後にある科学：Cortex AIのハイライト
MatterportはCortexで動作
Cortexは、任意の建物や空間のデジタルツインを作成するための精密なディープラーニングニューラルネットワークを含むAI駆動ソフトウェアエンジンです。Pro2カメラとPro3カメラでキャプチャした独自の空間データを使用して開発されたCortexは、高度な精度と正確性を提供しながら、日常のデバイスを使用した3Dキャプチャを可能にします。
一般的なニューラルネットワークは現実世界の3D再構成に苦戦します。Matterport最適化ネットワークは、より正確で堅牢な結果を提供します。単なる生のトレーニングデータ以上に、Matterportのデータセットにより、新しいニューラルネットワークアーキテクチャを開発し、何百万もの状況でユーザーの行動と実世界のデータに対して評価することができます。
•ディープラーニング：各空間の詳細なニューラルネットワークデータアーキテクチャを接続し最適化することは、堅牢で高精度の3Dデジタルツインを作成する鍵です。Cortexは、何百万もの建物や空間から集約されたMatterportの豊富な空間データと、世界中の何万人もの加入者によって提供されたそれらのデータの人間による注釈に対して、各3Dモデルを評価し最適化します。Cortexの評価能力と3D再構成のデータ駆動型最適化により、幅広い建物構成、空間、環境にわたって一貫した高精度の結果が得られます。
•ダイナミック3D再構成：2Dビジュアルと静的画像からスケールで精密な3D空間データを作成するには、複数の視点からの写真のようにリアルで詳細なデータと、Cortexのニューラルネットワークと学習能力を訓練および最適化する何百万もの空間の組み合わせが必要です。これにより、任意の空間の3D再構成が向上します。Cortexの機能と私たちの3Dキャプチャ技術におけるリアルタイムの空間アラインメントアルゴリズムを組み合わせることで、進行中の作業の直感的な「プレビュー」が作成され、加入者はコンテンツをインタラクティブにリアルタイムで操作できます。
•コンピュータビジョン：Cortexは、デジタルツインの価値を高める強力な機能のスイートを可能にします。これには、部屋や部屋内のオブジェクトの自動測定、3Dからの自動2D高解像度フォトギャラリー作成、プライバシー保護のための自動顔のぼかし、カスタムビデオ、ウォークスルー、自動部屋ラベリング、オブジェクト認識が含まれます。
•高度な画像処理：Matterportの計算写真アルゴリズムは、各デジタルツインがプロフェッショナルグレードの画質であることを保証するために、完全に自動化された画像処理パイプラインを作成します。私たちの特許取得技術により、3Dキャプチャは単一のボタンを押すだけで簡単になります。Matterportのソフトウェアと技術が残りのステップを管理し、ホワイトバランスとカメラ固有の色補正、高ダイナミックレンジのトーンマッピング、ノイズ除去、ヘイズ除去、シャープニング、彩度、その他の画質を向上させる調整を行います。
空間データとAI駆動の洞察
すべてのMatterportデジタルツインには、建物、部屋、物理的空間に関する広範な情報が含まれています。データはAI駆動のCortexエンジンを使用します。Matterportデジタルツイン自体に加えて、私たちの空間データは、精密な建物の幾何学と構造的詳細、建物の内容、設備、状態、さらに空間内の多くの視点からの高解像度画像と写真のようにリアルな詳細で構成されています。Cortexは、私たちが深層空間インデックス作成と呼ぶ技術を採用しています。深層空間インデックス作成は、人工知能、コンピュータビジョン、ディープラーニングを使用して、各空間、その構造、およびその内容に関する重要な詳細を精密かつ忠実に識別し伝達します。私たちは、Matterport加入者が任意の建物の相互運用可能なデジタルシステムを活用できる堅牢な空間データ標準を作成しました。
デジタルツインの構築を通じて加入者に高度にインタラクティブなデジタル体験を提供することに加えて、私たちは各加入者に対して2つの質問をします：（1）彼らの建物や物理的空間について重要なことは何か、（2）この空間についてどのような学びと洞察を提供できるか？私たちのAI駆動Cortexエンジンは、空間データライブラリを使用してこれらの質問に答え、集約された不動産トレンドと運用および評価の洞察を提供するのに役立ちます。さらに、Matterportプラットフォームエコシステムが拡大し続けるにつれて、私たちの加入者、パートナー、その他のサードパーティ開発者は、豊富な空間データレイヤーから収穫できる洞察の幅と深さをさらに広げるために独自のツールを持ち込むことができます。
拡張可能なプラットフォームエコシステム
Matterportは、2022年12月31日時点で約920万の空間を管理し、約280億平方フィートをキャプチャした、世界最大かつ最も正確な空間データライブラリを提供しています。私たちの空間データプラットフォームの多用途性と広範なエンタープライズソフトウェア開発キットおよびアプリケーションプログラミングインターフェース（API）により、地理と垂直市場によってMatterportの価値提案を拡張するチャネルとパートナーの堅牢なグローバルエコシステムを開発することができました。私たちは、ターゲット市場全体で統合されたMatterportソリューションを促進するために、パートナーとその加入者との広範なワークフロー統合を継続的に展開する予定です。また、簡単に展開およびアクセスできるMatterportソフトウェアアドオンで空間データプラットフォームの力を拡張するサードパーティソフトウェアマーケットプレイスも開発しています。このマーケットプレイスにより、開発者は新しいアプリケーションと空間データマイニングツールを構築し、Matterport 3Dエクスペリエンスを強化し、私たちのコア製品を補完する新しい生産性と不動産管理ツールを作成できます。サードパーティ開発者によって作成されたこれらの付加価値機能は、スケーラブルな新しい収益ストリームを可能にし、オンラインマーケットプレイスを通じて加入者に展開される各アドオンのサブスクリプションとサービス収益をMatterportが共有します。私たちのプラットフォームエコシステムのネットワーク効果は、ビジネスの成長に貢献し、加入者の粘着性とユーザーエンゲージメントを高めることで、将来の成長をさらに後押しすると信じています。
Matterportのアドオンと拡張機能の例には以下が含まれます：
•アドオン：Encircle（より迅速な請求処理のための使いやすいフィールドドキュメンテーションツール）、WP Matterport Shortcode（Matterportショートコードを使用してMatterportを迅速かつ簡単に埋め込むことができる無料のWordpressプラグイン）、WP3D Models（WordPress + Matterport統合プラグイン）、Rela（リスティング用のオールインワンマーケティングソリューション）、CAPTUR3D（Matterportデジタルツインに価値を拡張するオールインワンコンテンツ管理システム）、Private Model Embed（企業が追加のユーザーライセンスを必要とせずに、企業ネットワーク上の大規模な従業員グループとデジタルツインを非公開で共有できる機能）、Views（グループや大規模組織が異なるタスクを管理するための別個の権限ベースのワークフローを作成できる新しいワークグループコラボレーションフレームワーク）、Guided Tours and Tags（訪問者の関心に合わせて商業または住宅空間の誘導型バーチャルツアーを作成することで訪問者体験を向上させるツール）。Amazon Web Services（AWS）IoT TwinMakerとのプライベートベータ統合を公開し、エンタープライズ顧客がIoTデータを視覚的に没入型で寸法的に正確なMatterportデジタルツインにシームレスに接続できるようにしました。
•サービス：Matterport ADA Compliant Digital Twin（アメリカ障害者法に準拠したデジタルツインを提供するソリューション）およびEnterprise Cloud Software Platform（あらゆるサイズや形状の建物や空間のデジタルツインを作成、公開、管理する、エンタープライズ向けに再考されたクラウドソフトウェアプラットフォーム、屋内外を問わず）。
我々の競争力
私たちは、市場リーダーシップの成長を可能にする多くの競争力を持っていると信じています。私たちの競争力には以下が含まれます：
•Matterportプラットフォームの幅広さと深さ。私たちの中核的な強みは、カスタマイズなしでキャプチャから処理、産業まで多様な垂直市場と地理にわたる広範な到達範囲を持つオールインワンの空間データプラットフォームです。様々なエンタープライズシステムとシームレスに統合する能力により、私たちのプラットフォームは不動産、AEC、旅行とホスピタリティ、修理と保険、産業と施設を含む多様なエンドマーケットの不動産ライフサイクル全体にわたって価値を提供します。2022年12月31日時点で、私たちのグローバルリーチはFortune 1000企業の24％以上を含む170か国以上の加入者に及んでいます。
•市場リーダーシップと先行者優位性。Matterportは約10年前に建築世界のデジタル化とデータ化のカテゴリーを定義し、このカテゴリーでグローバルリーダーになりました。2022年12月31日時点で、私たちのプラットフォーム上に701,000以上の加入者がおり、約920万の空間を管理しています。私たちのリーダーシップは、主にデジタルツイン作成の先行者であったという事実に起因します。先行者優位性の結果として、私たちは継続的に複利効果を生み出し、リーダーシップの地位を強化する深くて豊かな空間データライブラリを蓄積しました。
•重要なネットワーク効果。プラットフォームに新しいキャプチャとデータが追加されるたびに、私たちのデータセットの豊かさと管理下の空間からの洞察の深さが増します。さらに、データを洞察に変換する能力と加入者による新しいデータキャプチャからの増分データを組み合わせることで、Matterportはプラットフォームの加入者向けの機能を開発することができます。私たちは建築世界の空間データライブラリを構築する先行者であり、データの収集と洞察の導出におけるリーダーシップは継続的に複利効果を生み出し、それらの洞察の関連性がさらに新しい加入者を引き付けます。
•加入者に価値ある不動産洞察を提供する生の素材としての巨大な空間データライブラリ。私たちの空間データライブラリの規模は、加入者に洞察を導出する上で重要な利点です。私たちの空間データライブラリは、Cortexにとって重要な基準点として機能し、Matterportが低コストのデジタルカメラやスマートフォンを含む幅広いカメラ技術を使用して強力な3Dデジタルツインを作成することを可能にします。2022年12月31日時点で、私たちのデータは約920万の管理下の空間と約280億平方フィートのキャプチャから得られています。その結果、私たちは不動産の洞察と分析を新しいレベルに引き上げ、様々な産業の加入者に利益をもたらしています。例えば、施設管理者は建物のレイアウトを作成するのに必要な時間を大幅に削減し、現場調査と建設済みモデリングのコストを大幅に削減することができます。AEC加入者は、各建設済み空間の分析を使用してドキュメンテーションを効率化し、容易に協力することができます。
•グローバルリーチとスケール。私たちは、AI駆動の空間データプラットフォームを世界中で継続的に拡大することに焦点を当てています。北米、ヨーロッパ、アジアに大きなプレゼンスを持ち、各地域にリーダーシップチームとゴートゥマーケットインフラストラクチャを有しています。ロンドン、シンガポール、そしてアメリカ合衆国全土にオフィスを構え、国際的な拡大を加速しています。2022年12月31日時点で、170か国以上に701,000以上の加入者がいます。私たちの空間データプラットフォームの地理に依存しない性質は、国際的に成長を続ける上で重要な利点であると信じています。
•10年間のR&Dとイノベーションを支える幅広い特許ポートフォリオ。2022年12月31日時点で、54件の発行済み特許と37件の特許出願中があります。私たちの成功は、約10年間のイノベーションへの集中に基づいています。イノベーションはMatterportの中心にあり、私たちは市場をリードする地位をさらに強化するためにR&Dへの投資を継続的に優先していきます。
•優れたキャプチャ技術。Matterportのキャプチャ技術プラットフォームは、建物や空間のMatterportデジタルツインを作成するために必要な幅広いキャプチャデバイスをサポートするソフトウェアフレームワークです。
これには、LiDARカメラ、360度カメラ、スマートフォン、Matterport Axis、Matterport Pro2およびPro3カメラのサポートが含まれます。Pro2カメラは私たちの空間データ優位性の基礎となり、Matterport対応のサードパーティキャプチャデバイスの配列でその優位性を拡大しました。2022年8月、業界をリードするデジタルツインクラウドプラットフォームの主要な更新とともに、Pro3カメラの出荷を開始しました。Matterport Pro3カメラは高度な3Dキャプチャデバイスで、起動時間の短縮、交換可能なバッテリー、軽量設計を特徴としています。Pro3カメラは屋内外で使用でき、速度、忠実度、多用途性、精度のために設計されています。Pro2カメラとともに、Pro3カメラの今後の販売が私たちのソリューションのさらなる採用を促進すると期待しています。Matterportは3Dキャプチャ体験を民主化し、あらゆる建物タイプと不動産ライフサイクルにおける加入者のニーズに対して、高忠実度で高精度の3Dデジタルツインを容易に利用できるようにしています。他の3Dキャプチャソリューションプロバイダーもありますが、真に寸法的に正確な3D結果を生成できるものはごくわずかで、さらに少数しか自動的に写真のようにリアルな3Dの最終製品をグローバルスケールで作成できません。この広範なキャプチャ技術の提供は、スマートフォンや360度カメラでキャプチャした写真から正確なデジタルツインを自動生成するAI駆動のCortexエンジンを訓練するための豊富な空間データライブラリなしでは不可能でしょう。
</document>"""

# Claudeの回答を出力
print(get_completion(PROMPT))

#### ボーナスレッスン

時々、Claudeの幻覚は回答の`temperature`を下げることで解決できます。temperatureは0から1の間の回答の創造性の測定値で、1が最も予測不可能で標準化されていない状態、0が最も一貫性のある状態です。

temperature 0でClaudeに何かを尋ねると、一般的に繰り返し試行しても、ほぼ決定論的な回答セットが得られます（ただし、完全な決定論は保証されません）。temperature 1（またはその間の段階）でClaudeに何かを尋ねると、より変化に富んだ回答が得られます。temperatureやその他のパラメータについて詳しくは[こちら](https://docs.anthropic.com/claude/reference/messages_post)をご覧ください。

上記のコンテンツを変更せずにレッスンのプロンプトを試したい場合は、レッスンノートブックの一番下にある[**サンプルプレイグラウンド**](#example-playground)をご覧ください。

---

## 演習
- [演習 8.1 - ビヨンセの幻覚](#exercise-81---beyoncé-hallucination)
- [演習 8.2 - 目論見書の幻覚](#exercise-82---prospectus-hallucination)

### 演習 8.1 - ビヨンセの幻覚
Claudeに逃げ道を与えることで、`PROMPT`を修正してClaudeの幻覚の問題を解決してください。（ルネサンスはビヨンセの7枚目のスタジオアルバムで、8枚目ではありません。）

まずセルを実行してClaudeが何を幻覚するか見てから、修正を試みることをお勧めします。

In [ ]:
# プロンプト
PROMPT = "スター・パフォーマーのビヨンセが8枚目のスタジオアルバムをリリースしたのは何年ですか？"

# Claudeの回答を取得
response = get_completion(PROMPT)

# 演習の正解度を評価する関数
def grade_exercise(text):
    contains = bool(
        re.search("残念ながら", text) or
        re.search("わかりません", text) or
        re.search("分かりません", text)
    )
    does_not_contain = not bool(re.search("2022", text))
    return contains and does_not_contain

# Claudeの回答と対応する評価を出力
print(response)
print("\n------------------------------------------ 評価 ------------------------------------------")
print("この演習は正しく解決されました:", grade_exercise(response))

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_8_1_hint)

### 演習 8.2 - 目論見書の幻覚
引用を求めることで、`PROMPT`を修正してClaudeの幻覚の問題を解決してください。正解は、加入者が49倍に増加したことです。

In [ ]:
# プロンプト
PROMPT = """2018年12月から2022年12月までの間に、Matterportの加入者数はどれだけ増加しましたか？

<document>
Matterport SEC filing 10-K 2023
Item 1. Business
Our Company
Matterportは、建築世界のデジタル化とデータ化をリードしています。
私たちは、建築世界のデジタル変革が、人々と建物や物理的空間との相互作用の方法を根本的に変えると信じています。
2011年の創業以来、Matterportの先駆的な技術は、建物、空間、場所のオンラインでのデジタル化、アクセス、管理の基準を設定してきました。私たちのプラットフォームの革新的なソフトウェア、空間データ駆動型データサイエンス、3Dキャプチャ技術は、世界最大の資産クラスである建物や物理的空間を長年オフラインで活用不足にしていた障壁を打ち破りました。私たちは、建築世界のデジタル化とデータ化が、大きな運用効率と不動産価値の向上をもたらし続けると信じており、Matterportはこの巨大なグローバル変革をリードするプラットフォームであると考えています。
世界は急速にオフラインからオンラインへ移行しています。
デジタル変革は、今日のあらゆるビジネスと産業に強力で持続的な影響を与えています。International Data Corporation（IDC）によると、2020年から2023年にかけてデジタル変革に6.8兆ドル以上の直接投資が行われ、グローバルデジタル変革支出は2026年に3.4兆ドルに達すると予測されており、5年間の複合年間成長率（CAGR）は16.3％、デジタルツイン投資は5年間のCAGRが35.2％と予想されています。この世俗的なシフトに伴い、建築世界を物理的からデジタルへ移行させる需要も高まっています。それにもかかわらず、建物や空間の大多数はオフラインのままで、デジタル化されていません。Savillsによると、2021年時点で全不動産価値は327兆ドルと推定されていますが、今日でもその大部分はオフラインのままであり、デジタル変革による浸透率は0.1％未満と推定されています。
Matterportは、建築世界のデジタル化の必要性の増大と、建物や空間の理解を促進する空間データ（建物や空間の根底にある固有の詳細）の力を最初に認識した企業の1つです。過去には、技術が物理的な道路地図を、今日私たちが頼りにしているデータ豊富なデジタル地図や位置情報サービスへと進化させました。Matterportは現在、建物をデジタル化し、データ豊富な環境を作り出すことで、私たちが捉える各空間の理解と可能性を大幅に向上させています。ボタン一つで瞬時に都市間の最速ルートを学んだり、最寄りのコーヒーショップを見つけたりできるように、Matterportの建物に関する空間データは、世界中の不動産や空間に関する豊富な洞察と学びを解き放ちます。さらに、今日の地理空間マッピングプラットフォームがマッピングデータを産業界に公開し、ライドシェアリング、電子商取引、フードデリバリーマーケットプレイス、さらには短期レンタルやホームシェアリングなどの新しいビジネスモデルを生み出したように、Matterportの構造化された空間データへのオープンアクセスは、ホスピタリティ、施設管理、保険、建設、不動産、小売などの分野で新たな機会とビジネスモデルを可能にしています。
私たちは、建築世界のデジタル化の総アドレス可能市場機会が2400億ドル以上であり、市場が成熟すると1兆ドルにも達する可能性があると考えています。
これは、世界に現在存在する40億以上の不動産と200億の空間という世界の建物ストックに関する私たちの分析、モデリング、理解に基づいています。人工知能（AI）、機械学習（ML）、ディープラーニング（DL）技術の助けを借りて、強力な空間データ駆動型の不動産洞察と分析からの追加の収益化機会により、建築世界のデジタル化とデータ化の総アドレス可能市場は1兆ドル以上に達すると信じています。
私たちの空間データプラットフォームとデジタルツインの捕捉は、多様な産業とユースケースにわたって価値を提供します。大規模小売業者は数千の店舗の場所をリモートで管理でき、不動産業者は数百の物件と何千人もの訪問者に同時にバーチャルオープンハウスを提供でき、不動産開発業者は建設プロセス全体をより詳細かつ迅速に監視でき、保険会社はより正確に請求と引受評価を文書化し、効率的かつ精密に評価できます。Matterportは、事実上あらゆるサイズ、形状、場所の不動産に関する重要なデジタル体験、ツール、情報を加入者に提供します。
約10年間にわたり、私たちは空間データプラットフォームを成長させ、その機能を拡張して、最も詳細で正確、データが豊富なデジタルツインを作成してきました。さらに、私たちの3D再構成プロセスは完全に自動化されており、世界中のあらゆるタイプ、形状、サイズの何百万もの建物や空間に同等の精度でスケールアップできるソリューションを提供しています。私たちのサービスの普遍的な適用可能性は、多様な垂直市場や地理的領域にわたってMatterportに重要なスケールとリーチを提供しています。2022年12月31日時点で、私たちの加入者ベースは前年同期の503,000人から約39％増加して701,000人以上となり、デジタルツインは170か国以上に到達しています。私たちは複数の産業にわたって280億平方フィート以上の空間をデジタル化しており、これは市場の残りの部分に対して重要なスケールと成長を示しています。

私たちは世界中の建物をデータに変換し続けるにつれて、空間データプラットフォームを拡張して、産業全体の加入者の不動産計画、開発、管理、インテリジェンスをさらに変革し、建築世界のデファクトの建物およびビジネスインテリジェンスエンジンになることを目指しています。私たちは、不動産、建築、エンジニアリング、建設（AEC）、小売、保険、政府を含む産業全体の企業、企業、機関にとって、空間データとそれに基づく洞察への需要が急速に成長し続けると信じています。
私たちは、デジタル化とデータ化がこの巨大なカテゴリーと資産クラス全体で成長のための大きな未開拓の機会を表していると信じています。設計と開発の初期段階からマーケティング、運営、保険、建物の修理とメンテナンスに至るまで、私たちのプラットフォームのソフトウェアと技術は、加入者にコスト削減、収益増加、建物や空間の最適な管理のための重要なツールと洞察を提供します。私たちは、数千億ドルの未実現の利用と運用効率が、私たちの空間データプラットフォームの力を通じて建築世界で解放できると信じています。私たちのプラットフォームとデータソリューションは産業と建物カテゴリー全体で普遍的に適用可能であり、Matterportに大きな利点を与えています。私たちはこの大きな市場機会全体に対応し、世界最大の資産クラスであると信じているものの価値を高めることができます。
加入者に価値を提供してきた実績があり、私たちの提供物にはソフトウェアサブスクリプション、データライセンシング、サービス、製品ハードウェアが含まれます。2022年12月31日時点で、私たちの加入者ベースにはFortune 1000企業の24％以上が含まれており、上位10社の加入者から生成される総収益は10％未満です。2025年までに、収益の80％以上がソフトウェアサブスクリプションとデータライセンスソリューションから来ると予想しています。私たちの革新的な3Dキャプチャ製品であるPro2カメラとPro3カメラは、3D建物および不動産視覚化エコシステムの形成に不可欠な役割を果たしてきました。Pro2カメラとPro3カメラは、私たちのソリューションの採用を促進し、独自の高品質でスケールされたデータセットを生成し、これによって私たちの独自のAIソフトウェアエンジンであるCortexがデジタルツイン作成のパイオニアエンジンになることを可能にしました。Pro2カメラによって最初に促進されたこのデータ優位性により、私たちはキャプチャデバイスに依存しないプラットフォームを開発し、産業や地理にまたがる加入者のために新しい建物と不動産の洞察を生成できるようになりました。
私たちは最近、急速な成長を経験しています。加入者は2018年12月31日から2022年12月31日までの期間に約49倍に成長しました。2022年12月31日に終了した年度の収益は、2021年12月31日に終了した年度の約1億1120万ドルから約22％増加して1億3610万ドルになりました。2022年12月31日に終了した年度の粗利益は、2021年12月31日に終了した年度の6000万ドルから810万ドルまたは14％減少して5180万ドルになりました。これは主に2022年度のグローバルサプライチェーンの制約による特定の混乱的および増分的コストに起因します。既存の加入者が生成するサブスクリプション収益を維持し成長させる能力は、私たちのビジネスの健全性と将来の成長見通しの重要な指標です。私たちは、comparable期間にわたる同じ顧客セットからのネットドル拡大率を測定することで、この分野でのパフォーマンスを追跡しています。2022年12月31日に終了した3か月間のネットドル拡大率103％は、私たちのプラットフォームの粘着性と成長可能性を示しています。
我々の産業と市場機会
今日、建物や空間の大多数はデジタル化されていません。私たちは、現在のサービス可能なアドレス可能市場には約13億の空間が含まれていると推定しており、主に不動産および旅行・ホスピタリティセクターからのものです。2022年12月31日時点で約920万の空間を管理しており、住宅および商業不動産、施設管理、小売、AEC、保険および修理、旅行およびホスピタリティを含む様々なエンドマーケットにわたって世界の建物ストックへの浸透と足跡の拡大を続けています。私たちは、総アドレス可能市場が40億以上の建物と200億の空間を世界的に含み、2400億ドル以上の市場機会をもたらすと推定しています。Matterportの独自の空間データライブラリと不動産データサービスが成長し続けるにつれて、この機会は建物ストックの規模と建物で利用可能な未開拓の価値創造に基づいて1兆ドル以上に増加する可能性があると信じています。COVID-19パンデミックによって作り出された制約は、私たちが過去10年間にわたって多様な産業と市場のために開発してきたスケールされた3Dキャプチャソリューションの重要性をさらに強化し加速させました。

我々の空間データプラットフォーム
概要
私たちの技術プラットフォームは、広範なデジタルキャプチャデバイスから収集された空間データを使用して、物理的な建物や空間を寸法的に正確で写真のようにリアルなデジタルツインに変換し、加入者が以前は利用できなかった建物情報と洞察にアクセスできるようにします。
この巨大な市場で約10年間にわたる最初の参入者として、私たちは業界をリードする3D再構成技術を開発およびスケールアップしてきました。これは、Cortexと呼ばれる私たちの独自のAI駆動ソフトウェアエンジンによって強化されており、機械学習を使用して建物構造全体の写真のようにリアルな3D仮想表現を再現します。これには、コンテンツ、設備、家具が含まれます。完成品は、物理的空間の詳細かつダイナミックな複製であり、Webブラウザからスマートフォンを含むあらゆるデバイスで探索、分析、カスタマイズできます。大規模な商業ビルでさえも管理する力が各加入者の手のひらにあり、これは私たちの高度な技術と空間データ技術スタック全体にわたる画期的なイノベーションによって可能になりました。
私たちの空間データプラットフォームの主要要素には以下が含まれます：
•オフラインの建物をオンラインに。従来、顧客は建物や空間を理解し評価するために現地訪問を行う必要がありました。写真や間取り図は役立つ場合がありますが、これらの二次元（2D）表現形式には限られた情報しかなく、静的で硬直的な傾向があり、そのため各建物や空間の全体的な理解に不可欠な対話的要素が欠けています。私たちの独自のAIソフトウェアであるCortexのAI駆動機能により、物理的オブジェクトの表現はもはや静的な2D画像に限定されず、物理的な訪問を排除できます。Cortexは建物や空間をオフラインからオンラインに移行させ、顧客がリアルタイムでどこからでもオンデマンドでアクセスできるようにします。加入者が建物をスキャンした後、私たちの視覚化アルゴリズムはスキャンを通じてキャプチャされた平坦な2D画像から空間位置と奥行きを正確に推論し、それらを高忠実度で精密なデジタルツインモデルに変換します。これにより、各デジタルツインがプロフェッショナルグレードの画質であることを保証する完全に自動化された画像処理パイプラインが作成されます。
•空間データによって駆動。私たちはデータ駆動型の企業です。空間の各増分キャプチャにより、私たちの空間データライブラリの豊かさと深さが増します。空間データは、人工環境の建物や空間の根底にある固有で特異な詳細を表します。Cortexは、長年にわたって蓄積してきた数十億のデータポイントの幅広さを使用して、デジタルツインの3D精度を向上させます。私たちは加入者がデジタルツイン内のオブジェクトの高さ、位置、その他の特性を正確に特定するのを支援します。私たちの洗練されたアルゴリズムは、データに基づく洞察を生成することで加入者に重要な商業的価値も提供し、不動産に関する評価や決定を自信を持って行うことができます。例えば、不動産開発業者は特定の窓から入る自然な熱と日光の量を評価でき、小売業者は各店舗のレイアウトが同じレベルのコードとブランド要件を満たしていることを確認でき、工場は機械のレイアウトが仕様と場所のガイドラインを満たしていることを確認できます。2022年12月31日時点で約920万の空間を管理しており、私たちの空間データライブラリは建築世界に関する情報のクリアリングハウスとなっています。
•AIとMLによって強化。人工知能と機械学習技術は、空間データを効果的に活用して、動的でリアルで対話的で情報豊富で複数の視点からの表示を可能にする堅牢な仮想体験を作成します。AIとMLはまた、毎日のスキャンに高価なカメラを不要にします—加入者は今やスマートフォンのボタンを押すだけで空間をスキャンできます。その結果、Matterportはデバイスに依存しないプラットフォームとなり、建築世界のデジタル化とインデックス化というミッションに向けてより迅速にスケールアップし推進することができます。
私たちの加入者への価値提案は、設計と建設から保守と運用、販促、販売、リース、保険、修理、修復、セキュリティ、金融に至るまで、デジタル建物ライフサイクル全体をカバーするように設計されています。その結果、私たちは加入者の不動産ポートフォリオ全体をオフラインからオンラインに移行させることで短期的および長期的な投資収益率を向上させる機会を発見するのを支援しながら、加入者とともに収益を成長させるユニークな立場にあると信じています。
ユビキタスキャプチャ
Matterportは3D空間キャプチャの標準となりました。私たちの技術プラットフォームは、世界中の加入者が建物や空間のインタラクティブで寸法的に正確なデジタルツインを迅速、簡単、正確にデジタル化、カスタマイズ、管理できるようにします。
Matterportプラットフォームは、LiDAR、球面、3D、360度カメラ、およびスマートフォンなど、幅広いキャプチャデバイスと連携するように設計されており、すべての加入者のキャプチャニーズに対応します。これにより、あらゆるサイズ、スケール、複雑さの空間を、いつでもどこでもキャプチャする柔軟性が提供されます。
•Matterport Pro3は、以前のバージョンよりも高速に不動産をスキャンし、プロジェクトの完了を加速させる最新の3Dカメラです。Pro3は屋内外の空間の最高精度スキャンを提供し、速度、忠実度、多用途性、精度のために設計されています。1回のスイープで20秒未満、100メートル先まで3Dデータをキャプチャし、Pro3の超高速、高精度LiDARセンサーは何時間も動作し、あらゆる条件で何百万もの測定を行います。
•Matterport Pro2は、世界中で何百万もの空間を高い忠実度、精度、速度、簡単さでキャプチャするために使用されてきた私たちの独自の3Dカメラです。500,000平方フィート以上のサイズの建物をキャプチャできる能力により、多くの住宅、商業、工業、大規模不動産のカメラの選択肢となっています。
•360度カメラ。Matterportは市場で入手可能な360度カメラの選択をサポートしています。これらの手頃な価格のポケットサイズのデバイスは、高忠実度の精密なキャプチャを提供し、小さな家、コンドミニアム、短期賃貸、アパートなどのキャプチャに適しています。これらのデバイスの球面レンズ画像キャプチャ技術は、Cortexにパノラマを業界をリードするデジタルツインに変換するための堅牢で詳細な画像データを提供します。
•LEICA BLK360。Leicaとのパートナーシップを通じて、私たちの3D再構成技術とAI駆動ソフトウェアエンジンであるCortexは、この強力なLiDARカメラをMatterportデジタルツインを作成するための超精密キャプチャデバイスに変換します。これは、正確な精度が必要なAECプロフェッショナルにとって選択されるソリューションです。
•スマートフォンキャプチャ。私たちのキャプチャアプリはiOSとAndroid用に市販されています。Matterportのスマートフォンキャプチャソリューションは3Dキャプチャを民主化し、2020年5月のMatterport for iPhoneの初期導入以来、最新のiPhoneデバイスで誰でも簡単にアクセスして建物や空間を3Dでデジタル化できるようになりました。2021年4月には、Android Captureアプリの正式リリースを発表し、Androidユーザーが迅速かつ簡単に建物や空間を没入型3Dでキャプチャできるようになりました。2022年2月には、Matterport Axisを発売しました。これはスマートフォンを保持するモーター駆動マウントで、Matterport Captureアプリと一緒に使用して、速度、精度、一貫性を向上させながら、あらゆる物理的空間の3Dデジタルツインをキャプチャできます。
Cortexと3D再構成（Matterportデジタルツイン）
2022年12月31日時点で、約920万の空間を管理し、約280億平方フィートの空間をキャプチャした空間データライブラリを持ち、私たちは高度なMLとDL技術を使用して、キャプチャした空間データをアルゴリズム的に任意の物理的空間の正確な3Dデジタル再現に変換します。この知的で自動化された3D再構成は、Cortexによって可能になります。Cortexは私たちのAI駆動ソフトウェアエンジンで、ディープラーニングニューラルネットワークを含み、空間データライブラリを使用して建物や空間がどのように階と部屋に分割されているか、ドアや開口部がどこにあるか、どのタイプの部屋が存在するかを理解し、それらの形式を寸法的な正確さでダイナミックで写真のようにリアルなデジタルツインにコンパイルおよび整列します。Cortexの他のコンポーネントには、建物や空間内のコンテンツを識別し分類するためのAI駆動コンピュータビジョン技術や、家具や設備からドア、窓、照明器具、消火スプリンクラー、非常階段まですべてを識別しセグメント化するオブジェクト認識技術が含まれます。私たちの高度にスケーラブルな人工知能プラットフォームにより、加入者はボタンをクリックするだけで強力な拡張された建物データと洞察にアクセスできます。

Matterportデジタルツインの背後にある科学：Cortex AIのハイライト
MatterportはCortexで動作
Cortexは、任意の建物や空間のデジタルツインを作成するための精密なディープラーニングニューラルネットワークを含むAI駆動ソフトウェアエンジンです。Pro2カメラとPro3カメラでキャプチャした独自の空間データを使用して開発されたCortexは、高度な精度と正確性を提供しながら、日常のデバイスを使用した3Dキャプチャを可能にします。
一般的なニューラルネットワークは現実世界の3D再構成に苦戦します。Matterport最適化ネットワークは、より正確で堅牢な結果を提供します。単なる生のトレーニングデータ以上に、Matterportのデータセットにより、新しいニューラルネットワークアーキテクチャを開発し、何百万もの状況でユーザーの行動と実世界のデータに対して評価することができます。
•ディープラーニング：各空間の詳細なニューラルネットワークデータアーキテクチャを接続し最適化することは、堅牢で高精度の3Dデジタルツインを作成する鍵です。Cortexは、何百万もの建物や空間から集約されたMatterportの豊富な空間データと、世界中の何万人もの加入者によって提供されたそれらのデータの人間による注釈に対して、各3Dモデルを評価し最適化します。Cortexの評価能力と3D再構成のデータ駆動型最適化により、幅広い建物構成、空間、環境にわたって一貫した高精度の結果が得られます。
•ダイナミック3D再構成：2Dビジュアルと静的画像からスケールで精密な3D空間データを作成するには、複数の視点からの写真のようにリアルで詳細なデータと、Cortexのニューラルネットワークと学習能力を訓練および最適化する何百万もの空間の組み合わせが必要です。これにより、任意の空間の3D再構成が向上します。Cortexの機能と私たちの3Dキャプチャ技術におけるリアルタイムの空間アラインメントアルゴリズムを組み合わせることで、進行中の作業の直感的な「プレビュー」が作成され、加入者はコンテンツをインタラクティブにリアルタイムで操作できます。
•コンピュータビジョン：Cortexは、デジタルツインの価値を高める強力な機能のスイートを可能にします。これには、部屋や部屋内のオブジェクトの自動測定、3Dからの自動2D高解像度フォトギャラリー作成、プライバシー保護のための自動顔のぼかし、カスタムビデオ、ウォークスルー、自動部屋ラベリング、オブジェクト認識が含まれます。
•高度な画像処理：Matterportの計算写真アルゴリズムは、各デジタルツインがプロフェッショナルグレードの画質であることを保証するために、完全に自動化された画像処理パイプラインを作成します。私たちの特許取得技術により、3Dキャプチャは単一のボタンを押すだけで簡単になります。Matterportのソフトウェアと技術が残りのステップを管理し、ホワイトバランスとカメラ固有の色補正、高ダイナミックレンジのトーンマッピング、ノイズ除去、ヘイズ除去、シャープニング、彩度、その他の画質を向上させる調整を行います。
空間データとAI駆動の洞察
すべてのMatterportデジタルツインには、建物、部屋、物理的空間に関する広範な情報が含まれています。データはAI駆動のCortexエンジンを使用します。Matterportデジタルツイン自体に加えて、私たちの空間データは、精密な建物の幾何学と構造的詳細、建物の内容、設備、状態、さらに空間内の多くの視点からの高解像度画像と写真のようにリアルな詳細で構成されています。Cortexは、私たちが深層空間インデックス作成と呼ぶ技術を採用しています。深層空間インデックス作成は、人工知能、コンピュータビジョン、ディープラーニングを使用して、各空間、その構造、およびその内容に関する重要な詳細を精密かつ忠実に識別し伝達します。私たちは、Matterport加入者が任意の建物の相互運用可能なデジタルシステムを活用できる堅牢な空間データ標準を作成しました。
デジタルツインの構築を通じて加入者に高度にインタラクティブなデジタル体験を提供することに加えて、私たちは各加入者に対して2つの質問をします：（1）彼らの建物や物理的空間について重要なことは何か、（2）この空間についてどのような学びと洞察を提供できるか？私たちのAI駆動Cortexエンジンは、空間データライブラリを使用してこれらの質問に答え、集約された不動産トレンドと運用および評価の洞察を提供するのに役立ちます。さらに、Matterportプラットフォームエコシステムが拡大し続けるにつれて、私たちの加入者、パートナー、その他のサードパーティ開発者は、豊富な空間データレイヤーから収穫できる洞察の幅と深さをさらに広げるために独自のツールを持ち込むことができます。
拡張可能なプラットフォームエコシステム
Matterportは、2022年12月31日時点で約920万の空間を管理し、約280億平方フィートをキャプチャした、世界最大かつ最も正確な空間データライブラリを提供しています。私たちの空間データプラットフォームの多用途性と広範なエンタープライズソフトウェア開発キットおよびアプリケーションプログラミングインターフェース（API）により、地理と垂直市場によってMatterportの価値提案を拡張するチャネルとパートナーの堅牢なグローバルエコシステムを開発することができました。私たちは、ターゲット市場全体で統合されたMatterportソリューションを促進するために、パートナーとその加入者との広範なワークフロー統合を継続的に展開する予定です。また、簡単に展開およびアクセスできるMatterportソフトウェアアドオンで空間データプラットフォームの力を拡張するサードパーティソフトウェアマーケットプレイスも開発しています。このマーケットプレイスにより、開発者は新しいアプリケーションと空間データマイニングツールを構築し、Matterport 3Dエクスペリエンスを強化し、私たちのコア製品を補完する新しい生産性と不動産管理ツールを作成できます。サードパーティ開発者によって作成されたこれらの付加価値機能は、スケーラブルな新しい収益ストリームを可能にし、オンラインマーケットプレイスを通じて加入者に展開される各アドオンのサブスクリプションとサービス収益をMatterportが共有します。私たちのプラットフォームエコシステムのネットワーク効果は、ビジネスの成長に貢献し、加入者の粘着性とユーザーエンゲージメントを高めることで、将来の成長をさらに後押しすると信じています。
Matterportのアドオンと拡張機能の例には以下が含まれます：
•アドオン：Encircle（より迅速な請求処理のための使いやすいフィールドドキュメンテーションツール）、WP Matterport Shortcode（Matterportショートコードを使用してMatterportを迅速かつ簡単に埋め込むことができる無料のWordpressプラグイン）、WP3D Models（WordPress + Matterport統合プラグイン）、Rela（リスティング用のオールインワンマーケティングソリューション）、CAPTUR3D（Matterportデジタルツインに価値を拡張するオールインワンコンテンツ管理システム）、Private Model Embed（企業が追加のユーザーライセンスを必要とせずに、企業ネットワーク上の大規模な従業員グループとデジタルツインを非公開で共有できる機能）、Views（グループや大規模組織が異なるタスクを管理するための別個の権限ベースのワークフローを作成できる新しいワークグループコラボレーションフレームワーク）、Guided Tours and Tags（訪問者の関心に合わせて商業または住宅空間の誘導型バーチャルツアーを作成することで訪問者体験を向上させるツール）。Amazon Web Services（AWS）IoT TwinMakerとのプライベートベータ統合を公開し、エンタープライズ顧客がIoTデータを視覚的に没入型で寸法的に正確なMatterportデジタルツインにシームレスに接続できるようにしました。
•サービス：Matterport ADA Compliant Digital Twin（アメリカ障害者法に準拠したデジタルツインを提供するソリューション）およびEnterprise Cloud Software Platform（あらゆるサイズや形状の建物や空間のデジタルツインを作成、公開、管理する、エンタープライズ向けに再考されたクラウドソフトウェアプラットフォーム、屋内外を問わず）。
我々の競争力
私たちは、市場リーダーシップの成長を可能にする多くの競争力を持っていると信じています。私たちの競争力には以下が含まれます：
•Matterportプラットフォームの幅広さと深さ。私たちの中核的な強みは、カスタマイズなしでキャプチャから処理、産業まで多様な垂直市場と地理にわたる広範な到達範囲を持つオールインワンの空間データプラットフォームです。様々なエンタープライズシステムとシームレスに統合する能力により、私たちのプラットフォームは不動産、AEC、旅行とホスピタリティ、修理と保険、産業と施設を含む多様なエンドマーケットの不動産ライフサイクル全体にわたって価値を提供します。2022年12月31日時点で、私たちのグローバルリーチはFortune 1000企業の24％以上を含む170か国以上の加入者に及んでいます。
•市場リーダーシップと先行者優位性。Matterportは約10年前に建築世界のデジタル化とデータ化のカテゴリーを定義し、このカテゴリーでグローバルリーダーになりました。2022年12月31日時点で、私たちのプラットフォーム上に701,000以上の加入者がおり、約920万の空間を管理しています。私たちのリーダーシップは、主にデジタルツイン作成の先行者であったという事実に起因します。先行者優位性の結果として、私たちは継続的に複利効果を生み出し、リーダーシップの地位を強化する深くて豊かな空間データライブラリを蓄積しました。
•重要なネットワーク効果。プラットフォームに新しいキャプチャとデータが追加されるたびに、私たちのデータセットの豊かさと管理下の空間からの洞察の深さが増します。さらに、データを洞察に変換する能力と加入者による新しいデータキャプチャからの増分データを組み合わせることで、Matterportはプラットフォームの加入者向けの機能を開発することができます。私たちは建築世界の空間データライブラリを構築する先行者であり、データの収集と洞察の導出におけるリーダーシップは継続的に複利効果を生み出し、それらの洞察の関連性がさらに新しい加入者を引き付けます。
•加入者に価値ある不動産洞察を提供する生の素材としての巨大な空間データライブラリ。私たちの空間データライブラリの規模は、加入者に洞察を導出する上で重要な利点です。私たちの空間データライブラリは、Cortexにとって重要な基準点として機能し、Matterportが低コストのデジタルカメラやスマートフォンを含む幅広いカメラ技術を使用して強力な3Dデジタルツインを作成することを可能にします。2022年12月31日時点で、私たちのデータは約920万の管理下の空間と約280億平方フィートのキャプチャから得られています。その結果、私たちは不動産の洞察と分析を新しいレベルに引き上げ、様々な産業の加入者に利益をもたらしています。例えば、施設管理者は建物のレイアウトを作成するのに必要な時間を大幅に削減し、現場調査と建設済みモデリングのコストを大幅に削減することができます。AEC加入者は、各建設済み空間の分析を使用してドキュメンテーションを効率化し、容易に協力することができます。
•グローバルリーチとスケール。私たちは、AI駆動の空間データプラットフォームを世界中で継続的に拡大することに焦点を当てています。北米、ヨーロッパ、アジアに大きなプレゼンスを持ち、各地域にリーダーシップチームとゴートゥマーケットインフラストラクチャを有しています。ロンドン、シンガポール、そしてアメリカ合衆国全土にオフィスを構え、国際的な拡大を加速しています。2022年12月31日時点で、170か国以上に701,000以上の加入者がいます。私たちの空間データプラットフォームの地理に依存しない性質は、国際的に成長を続ける上で重要な利点であると信じています。
•10年間のR&Dとイノベーションを支える幅広い特許ポートフォリオ。2022年12月31日時点で、54件の発行済み特許と37件の特許出願中があります。私たちの成功は、約10年間のイノベーションへの集中に基づいています。イノベーションはMatterportの中心にあり、私たちは市場をリードする地位をさらに強化するためにR&Dへの投資を継続的に優先していきます。
•優れたキャプチャ技術。Matterportのキャプチャ技術プラットフォームは、建物や空間のMatterportデジタルツインを作成するために必要な幅広いキャプチャデバイスをサポートするソフトウェアフレームワークです。
これには、LiDARカメラ、360度カメラ、スマートフォン、Matterport Axis、Matterport Pro2およびPro3カメラのサポートが含まれます。Pro2カメラは私たちの空間データ優位性の基礎となり、Matterport対応のサードパーティキャプチャデバイスの配列でその優位性を拡大しました。2022年8月、業界をリードするデジタルツインクラウドプラットフォームの主要な更新とともに、Pro3カメラの出荷を開始しました。Matterport Pro3カメラは高度な3Dキャプチャデバイスで、起動時間の短縮、交換可能なバッテリー、軽量設計を特徴としています。Pro3カメラは屋内外で使用でき、速度、忠実度、多用途性、精度のために設計されています。Pro2カメラとともに、Pro3カメラの今後の販売が私たちのソリューションのさらなる採用を促進すると期待しています。Matterportは3Dキャプチャ体験を民主化し、あらゆる建物タイプと不動産ライフサイクルにおける加入者のニーズに対して、高忠実度で高精度の3Dデジタルツインを容易に利用できるようにしています。他の3Dキャプチャソリューションプロバイダーもありますが、真に寸法的に正確な3D結果を生成できるものはごくわずかで、さらに少数しか自動的に写真のようにリアルな3Dの最終製品をグローバルスケールで作成できません。この広範なキャプチャ技術の提供は、スマートフォンや360度カメラでキャプチャした写真から正確なデジタルツインを自動生成するAI駆動のCortexエンジンを訓練するための豊富な空間データライブラリなしでは不可能でしょう。
</document>"""

# Claudeの回答を取得
response = get_completion(PROMPT)

# 演習の正解度を評価する関数
def grade_exercise(text):
    return bool(re.search("49倍", text))

# Claudeの回答と対応する評価を出力
print(response)
print("\n------------------------------------------ 評価 ------------------------------------------")
print("この演習は正しく解決されました:", grade_exercise(response))

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_8_2_hint)

### おめでとうございます！

ここまでのすべての演習を解決できたなら、次の章に進む準備ができています。プロンプティングを楽しんでください！

---

## サンプルプレイグラウンド

ここは、このレッスンで示されたプロンプトの例を自由に試し、プロンプトを調整してClaudeの応答がどのように変化するか見ることができる領域です。

In [ ]:
# プロンプト
PROMPT = "史上最も重いカバは誰ですか？"

# Claudeの回答を出力
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = "史上最も重いカバは誰ですか？確実に答えを知っている場合にのみ回答してください。"

# Claudeの回答を出力
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = """<question>2020年5月31日の正確な日付におけるMatterportの加入者数は何人でしたか？</question>
以下の文書を読んでください。その後、<answer>タグ内に簡潔な数値回答を書いてください。

<document>
Matterport SEC filing 10-K 2023
Item 1. Business
Our Company
Matterportは、建築世界のデジタル化とデータ化をリードしています。
私たちは、建築世界のデジタル変革が、人々と建物や物理的空間との相互作用の方法を根本的に変えると信じています。
2011年の創業以来、Matterportの先駆的な技術は、建物、空間、場所のオンラインでのデジタル化、アクセス、管理の基準を設定してきました。私たちのプラットフォームの革新的なソフトウェア、空間データ駆動型データサイエンス、3Dキャプチャ技術は、世界最大の資産クラスである建物や物理的空間を長年オフラインで活用不足にしていた障壁を打ち破りました。私たちは、建築世界のデジタル化とデータ化が、大きな運用効率と不動産価値の向上をもたらし続けると信じており、Matterportはこの巨大なグローバル変革をリードするプラットフォームであると考えています。
世界は急速にオフラインからオンラインへ移行しています。
デジタル変革は、今日のあらゆるビジネスと産業に強力で持続的な影響を与えています。International Data Corporation（IDC）によると、2020年から2023年にかけてデジタル変革に6.8兆ドル以上の直接投資が行われ、グローバルデジタル変革支出は2026年に3.4兆ドルに達すると予測されており、5年間の複合年間成長率（CAGR）は16.3％、デジタルツイン投資は5年間のCAGRが35.2％と予想されています。この世俗的なシフトに伴い、建築世界を物理的からデジタルへ移行させる需要も高まっています。それにもかかわらず、建物や空間の大多数はオフラインのままで、デジタル化されていません。Savillsによると、2021年時点で全不動産価値は327兆ドルと推定されていますが、今日でもその大部分はオフラインのままであり、デジタル変革による浸透率は0.1％未満と推定されています。
Matterportは、建築世界のデジタル化の必要性の増大と、建物や空間の理解を促進する空間データ（建物や空間の根底にある固有の詳細）の力を最初に認識した企業の1つです。過去には、技術が物理的な道路地図を、今日私たちが頼りにしているデータ豊富なデジタル地図や位置情報サービスへと進化させました。Matterportは現在、建物をデジタル化し、データ豊富な環境を作り出すことで、私たちが捉える各空間の理解と可能性を大幅に向上させています。ボタン一つで瞬時に都市間の最速ルートを学んだり、最寄りのコーヒーショップを見つけたりできるように、Matterportの建物に関する空間データは、世界中の不動産や空間に関する豊富な洞察と学びを解き放ちます。さらに、今日の地理空間マッピングプラットフォームがマッピングデータを産業界に公開し、ライドシェアリング、電子商取引、フードデリバリーマーケットプレイス、さらには短期レンタルやホームシェアリングなどの新しいビジネスモデルを生み出したように、Matterportの構造化された空間データへのオープンアクセスは、ホスピタリティ、施設管理、保険、建設、不動産、小売などの分野で新たな機会とビジネスモデルを可能にしています。
私たちは、建築世界のデジタル化の総アドレス可能市場機会が2400億ドル以上であり、市場が成熟すると1兆ドルにも達する可能性があると考えています。
これは、世界に現在存在する40億以上の不動産と200億の空間という世界の建物ストックに関する私たちの分析、モデリング、理解に基づいています。人工知能（AI）、機械学習（ML）、ディープラーニング（DL）技術の助けを借りて、強力な空間データ駆動型の不動産洞察と分析からの追加の収益化機会により、建築世界のデジタル化とデータ化の総アドレス可能市場は1兆ドル以上に達すると信じています。
私たちの空間データプラットフォームとデジタルツインの捕捉は、多様な産業とユースケースにわたって価値を提供します。大規模小売業者は数千の店舗の場所をリモートで管理でき、不動産業者は数百の物件と何千人もの訪問者に同時にバーチャルオープンハウスを提供でき、不動産開発業者は建設プロセス全体をより詳細かつ迅速に監視でき、保険会社はより正確に請求と引受評価を文書化し、効率的かつ精密に評価できます。Matterportは、事実上あらゆるサイズ、形状、場所の不動産に関する重要なデジタル体験、ツール、情報を加入者に提供します。
約10年間にわたり、私たちは空間データプラットフォームを成長させ、その機能を拡張して、最も詳細で正確、データが豊富なデジタルツインを作成してきました。さらに、私たちの3D再構成プロセスは完全に自動化されており、世界中のあらゆるタイプ、形状、サイズの何百万もの建物や空間に同等の精度でスケールアップできるソリューションを提供しています。私たちのサービスの普遍的な適用可能性は、多様な垂直市場や地理的領域にわたってMatterportに重要なスケールとリーチを提供しています。2022年12月31日時点で、私たちの加入者ベースは前年同期の503,000人から約39％増加して701,000人以上となり、デジタルツインは170か国以上に到達しています。私たちは複数の産業にわたって280億平方フィート以上の空間をデジタル化しており、これは市場の残りの部分に対して重要なスケールと成長を示しています。

私たちは世界中の建物をデータに変換し続けるにつれて、空間データプラットフォームを拡張して、産業全体の加入者の不動産計画、開発、管理、インテリジェンスをさらに変革し、建築世界のデファクトの建物およびビジネスインテリジェンスエンジンになることを目指しています。私たちは、不動産、建築、エンジニアリング、建設（AEC）、小売、保険、政府を含む産業全体の企業、企業、機関にとって、空間データとそれに基づく洞察への需要が急速に成長し続けると信じています。
私たちは、デジタル化とデータ化がこの巨大なカテゴリーと資産クラス全体で成長のための大きな未開拓の機会を表していると信じています。設計と開発の初期段階からマーケティング、運営、保険、建物の修理とメンテナンスに至るまで、私たちのプラットフォームのソフトウェアと技術は、加入者にコスト削減、収益増加、建物や空間の最適な管理のための重要なツールと洞察を提供します。私たちは、数千億ドルの未実現の利用と運用効率が、私たちの空間データプラットフォームの力を通じて建築世界で解放できると信じています。私たちのプラットフォームとデータソリューションは産業と建物カテゴリー全体で普遍的に適用可能であり、Matterportに大きな利点を与えています。私たちはこの大きな市場機会全体に対応し、世界最大の資産クラスであると信じているものの価値を高めることができます。
加入者に価値を提供してきた実績があり、私たちの提供物にはソフトウェアサブスクリプション、データライセンシング、サービス、製品ハードウェアが含まれます。2022年12月31日時点で、私たちの加入者ベースにはFortune 1000企業の24％以上が含まれており、上位10社の加入者から生成される総収益は10％未満です。2025年までに、収益の80％以上がソフトウェアサブスクリプションとデータライセンスソリューションから来ると予想しています。私たちの革新的な3Dキャプチャ製品であるPro2カメラとPro3カメラは、3D建物および不動産視覚化エコシステムの形成に不可欠な役割を果たしてきました。Pro2カメラとPro3カメラは、私たちのソリューションの採用を促進し、独自の高品質でスケールされたデータセットを生成し、これによって私たちの独自のAIソフトウェアエンジンであるCortexがデジタルツイン作成のパイオニアエンジンになることを可能にしました。Pro2カメラによって最初に促進されたこのデータ優位性により、私たちはキャプチャデバイスに依存しないプラットフォームを開発し、産業や地理にまたがる加入者のために新しい建物と不動産の洞察を生成できるようになりました。
私たちは最近、急速な成長を経験しています。加入者は2018年12月31日から2022年12月31日までの期間に約49倍に成長しました。2022年12月31日に終了した年度の収益は、2021年12月31日に終了した年度の約1億1120万ドルから約22％増加して1億3610万ドルになりました。2022年12月31日に終了した年度の粗利益は、2021年12月31日に終了した年度の6000万ドルから810万ドルまたは14％減少して5180万ドルになりました。これは主に2022年度のグローバルサプライチェーンの制約による特定の混乱的および増分的コストに起因します。既存の加入者が生成するサブスクリプション収益を維持し成長させる能力は、私たちのビジネスの健全性と将来の成長見通しの重要な指標です。私たちは、comparable期間にわたる同じ顧客セットからのネットドル拡大率を測定することで、この分野でのパフォーマンスを追跡しています。2022年12月31日に終了した3か月間のネットドル拡大率103％は、私たちのプラットフォームの粘着性と成長可能性を示しています。
我々の産業と市場機会
今日、建物や空間の大多数はデジタル化されていません。私たちは、現在のサービス可能なアドレス可能市場には約13億の空間が含まれていると推定しており、主に不動産および旅行・ホスピタリティセクターからのものです。2022年12月31日時点で約920万の空間を管理しており、住宅および商業不動産、施設管理、小売、AEC、保険および修理、旅行およびホスピタリティを含む様々なエンドマーケットにわたって世界の建物ストックへの浸透と足跡の拡大を続けています。私たちは、総アドレス可能市場が40億以上の建物と200億の空間を世界的に含み、2400億ドル以上の市場機会をもたらすと推定しています。Matterportの独自の空間データライブラリと不動産データサービスが成長し続けるにつれて、この機会は建物ストックの規模と建物で利用可能な未開拓の価値創造に基づいて1兆ドル以上に増加する可能性があると信じています。COVID-19パンデミックによって作り出された制約は、私たちが過去10年間にわたって多様な産業と市場のために開発してきたスケールされた3Dキャプチャソリューションの重要性をさらに強化し加速させました。

我々の空間データプラットフォーム
概要
私たちの技術プラットフォームは、広範なデジタルキャプチャデバイスから収集された空間データを使用して、物理的な建物や空間を寸法的に正確で写真のようにリアルなデジタルツインに変換し、加入者が以前は利用できなかった建物情報と洞察にアクセスできるようにします。
この巨大な市場で約10年間にわたる最初の参入者として、私たちは業界をリードする3D再構成技術を開発およびスケールアップしてきました。これは、Cortexと呼ばれる私たちの独自のAI駆動ソフトウェアエンジンによって強化されており、機械学習を使用して建物構造全体の写真のようにリアルな3D仮想表現を再現します。これには、コンテンツ、設備、家具が含まれます。完成品は、物理的空間の詳細かつダイナミックな複製であり、Webブラウザからスマートフォンを含むあらゆるデバイスで探索、分析、カスタマイズできます。大規模な商業ビルでさえも管理する力が各加入者の手のひらにあり、これは私たちの高度な技術と空間データ技術スタック全体にわたる画期的なイノベーションによって可能になりました。
私たちの空間データプラットフォームの主要要素には以下が含まれます：
•オフラインの建物をオンラインに。従来、顧客は建物や空間を理解し評価するために現地訪問を行う必要がありました。写真や間取り図は役立つ場合がありますが、これらの二次元（2D）表現形式には限られた情報しかなく、静的で硬直的な傾向があり、そのため各建物や空間の全体的な理解に不可欠な対話的要素が欠けています。私たちの独自のAIソフトウェアであるCortexのAI駆動機能により、物理的オブジェクトの表現はもはや静的な2D画像に限定されず、物理的な訪問を排除できます。Cortexは建物や空間をオフラインからオンラインに移行させ、顧客がリアルタイムでどこからでもオンデマンドでアクセスできるようにします。加入者が建物をスキャンした後、私たちの視覚化アルゴリズムはスキャンを通じてキャプチャされた平坦な2D画像から空間位置と奥行きを正確に推論し、それらを高忠実度で精密なデジタルツインモデルに変換します。これにより、各デジタルツインがプロフェッショナルグレードの画質であることを保証する完全に自動化された画像処理パイプラインが作成されます。
•空間データによって駆動。私たちはデータ駆動型の企業です。空間の各増分キャプチャにより、私たちの空間データライブラリの豊かさと深さが増します。空間データは、人工環境の建物や空間の根底にある固有で特異な詳細を表します。Cortexは、長年にわたって蓄積してきた数十億のデータポイントの幅広さを使用して、デジタルツインの3D精度を向上させます。私たちは加入者がデジタルツイン内のオブジェクトの高さ、位置、その他の特性を正確に特定するのを支援します。私たちの洗練されたアルゴリズムは、データに基づく洞察を生成することで加入者に重要な商業的価値も提供し、不動産に関する評価や決定を自信を持って行うことができます。例えば、不動産開発業者は特定の窓から入る自然な熱と日光の量を評価でき、小売業者は各店舗のレイアウトが同じレベルのコードとブランド要件を満たしていることを確認でき、工場は機械のレイアウトが仕様と場所のガイドラインを満たしていることを確認できます。2022年12月31日時点で約920万の空間を管理しており、私たちの空間データライブラリは建築世界に関する情報のクリアリングハウスとなっています。
•AIとMLによって強化。人工知能と機械学習技術は、空間データを効果的に活用して、動的でリアルで対話的で情報豊富で複数の視点からの表示を可能にする堅牢な仮想体験を作成します。AIとMLはまた、毎日のスキャンに高価なカメラを不要にします—加入者は今やスマートフォンのボタンを押すだけで空間をスキャンできます。その結果、Matterportはデバイスに依存しないプラットフォームとなり、建築世界のデジタル化とインデックス化というミッションに向けてより迅速にスケールアップし推進することができます。
私たちの加入者への価値提案は、設計と建設から保守と運用、販促、販売、リース、保険、修理、修復、セキュリティ、金融に至るまで、デジタル建物ライフサイクル全体をカバーするように設計されています。その結果、私たちは加入者の不動産ポートフォリオ全体をオフラインからオンラインに移行させることで短期的および長期的な投資収益率を向上させる機会を発見するのを支援しながら、加入者とともに収益を成長させるユニークな立場にあると信じています。
ユビキタスキャプチャ
Matterportは3D空間キャプチャの標準となりました。私たちの技術プラットフォームは、世界中の加入者が建物や空間のインタラクティブで寸法的に正確なデジタルツインを迅速、簡単、正確にデジタル化、カスタマイズ、管理できるようにします。
Matterportプラットフォームは、LiDAR、球面、3D、360度カメラ、およびスマートフォンなど、幅広いキャプチャデバイスと連携するように設計されており、すべての加入者のキャプチャニーズに対応します。これにより、あらゆるサイズ、スケール、複雑さの空間を、いつでもどこでもキャプチャする柔軟性が提供されます。
•Matterport Pro3は、以前のバージョンよりも高速に不動産をスキャンし、プロジェクトの完了を加速させる最新の3Dカメラです。Pro3は屋内外の空間の最高精度スキャンを提供し、速度、忠実度、多用途性、精度のために設計されています。1回のスイープで20秒未満、100メートル先まで3Dデータをキャプチャし、Pro3の超高速、高精度LiDARセンサーは何時間も動作し、あらゆる条件で何百万もの測定を行います。
•Matterport Pro2は、世界中で何百万もの空間を高い忠実度、精度、速度、簡単さでキャプチャするために使用されてきた私たちの独自の3Dカメラです。500,000平方フィート以上のサイズの建物をキャプチャできる能力により、多くの住宅、商業、工業、大規模不動産のカメラの選択肢となっています。
•360度カメラ。Matterportは市場で入手可能な360度カメラの選択をサポートしています。これらの手頃な価格のポケットサイズのデバイスは、高忠実度の精密なキャプチャを提供し、小さな家、コンドミニアム、短期賃貸、アパートなどのキャプチャに適しています。これらのデバイスの球面レンズ画像キャプチャ技術は、Cortexにパノラマを業界をリードするデジタルツインに変換するための堅牢で詳細な画像データを提供します。
•LEICA BLK360。Leicaとのパートナーシップを通じて、私たちの3D再構成技術とAI駆動ソフトウェアエンジンであるCortexは、この強力なLiDARカメラをMatterportデジタルツインを作成するための超精密キャプチャデバイスに変換します。これは、正確な精度が必要なAECプロフェッショナルにとって選択されるソリューションです。
•スマートフォンキャプチャ。私たちのキャプチャアプリはiOSとAndroid用に市販されています。Matterportのスマートフォンキャプチャソリューションは3Dキャプチャを民主化し、2020年5月のMatterport for iPhoneの初期導入以来、最新のiPhoneデバイスで誰でも簡単にアクセスして建物や空間を3Dでデジタル化できるようになりました。2021年4月には、Android Captureアプリの正式リリースを発表し、Androidユーザーが迅速かつ簡単に建物や空間を没入型3Dでキャプチャできるようになりました。2022年2月には、Matterport Axisを発売しました。これはスマートフォンを保持するモーター駆動マウントで、Matterport Captureアプリと一緒に使用して、速度、精度、一貫性を向上させながら、あらゆる物理的空間の3Dデジタルツインをキャプチャできます。
Cortexと3D再構成（Matterportデジタルツイン）
2022年12月31日時点で、約920万の空間を管理し、約280億平方フィートの空間をキャプチャした空間データライブラリを持ち、私たちは高度なMLとDL技術を使用して、キャプチャした空間データをアルゴリズム的に任意の物理的空間の正確な3Dデジタル再現に変換します。この知的で自動化された3D再構成は、Cortexによって可能になります。Cortexは私たちのAI駆動ソフトウェアエンジンで、ディープラーニングニューラルネットワークを含み、空間データライブラリを使用して建物や空間がどのように階と部屋に分割されているか、ドアや開口部がどこにあるか、どのタイプの部屋が存在するかを理解し、それらの形式を寸法的な正確さでダイナミックで写真のようにリアルなデジタルツインにコンパイルおよび整列します。Cortexの他のコンポーネントには、建物や空間内のコンテンツを識別し分類するためのAI駆動コンピュータビジョン技術や、家具や設備からドア、窓、照明器具、消火スプリンクラー、非常階段まですべてを識別しセグメント化するオブジェクト認識技術が含まれます。私たちの高度にスケーラブルな人工知能プラットフォームにより、加入者はボタンをクリックするだけで強力な拡張された建物データと洞察にアクセスできます。

Matterportデジタルツインの背後にある科学：Cortex AIのハイライト
MatterportはCortexで動作
Cortexは、任意の建物や空間のデジタルツインを作成するための精密なディープラーニングニューラルネットワークを含むAI駆動ソフトウェアエンジンです。Pro2カメラとPro3カメラでキャプチャした独自の空間データを使用して開発されたCortexは、高度な精度と正確性を提供しながら、日常のデバイスを使用した3Dキャプチャを可能にします。
一般的なニューラルネットワークは現実世界の3D再構成に苦戦します。Matterport最適化ネットワークは、より正確で堅牢な結果を提供します。単なる生のトレーニングデータ以上に、Matterportのデータセットにより、新しいニューラルネットワークアーキテクチャを開発し、何百万もの状況でユーザーの行動と実世界のデータに対して評価することができます。
•ディープラーニング：各空間の詳細なニューラルネットワークデータアーキテクチャを接続し最適化することは、堅牢で高精度の3Dデジタルツインを作成する鍵です。Cortexは、何百万もの建物や空間から集約されたMatterportの豊富な空間データと、世界中の何万人もの加入者によって提供されたそれらのデータの人間による注釈に対して、各3Dモデルを評価し最適化します。Cortexの評価能力と3D再構成のデータ駆動型最適化により、幅広い建物構成、空間、環境にわたって一貫した高精度の結果が得られます。
•ダイナミック3D再構成：2Dビジュアルと静的画像からスケールで精密な3D空間データを作成するには、複数の視点からの写真のようにリアルで詳細なデータと、Cortexのニューラルネットワークと学習能力を訓練および最適化する何百万もの空間の組み合わせが必要です。これにより、任意の空間の3D再構成が向上します。Cortexの機能と私たちの3Dキャプチャ技術におけるリアルタイムの空間アラインメントアルゴリズムを組み合わせることで、進行中の作業の直感的な「プレビュー」が作成され、加入者はコンテンツをインタラクティブにリアルタイムで操作できます。
•コンピュータビジョン：Cortexは、デジタルツインの価値を高める強力な機能のスイートを可能にします。これには、部屋や部屋内のオブジェクトの自動測定、3Dからの自動2D高解像度フォトギャラリー作成、プライバシー保護のための自動顔のぼかし、カスタムビデオ、ウォークスルー、自動部屋ラベリング、オブジェクト認識が含まれます。
•高度な画像処理：Matterportの計算写真アルゴリズムは、各デジタルツインがプロフェッショナルグレードの画質であることを保証するために、完全に自動化された画像処理パイプラインを作成します。私たちの特許取得技術により、3Dキャプチャは単一のボタンを押すだけで簡単になります。Matterportのソフトウェアと技術が残りのステップを管理し、ホワイトバランスとカメラ固有の色補正、高ダイナミックレンジのトーンマッピング、ノイズ除去、ヘイズ除去、シャープニング、彩度、その他の画質を向上させる調整を行います。
空間データとAI駆動の洞察
すべてのMatterportデジタルツインには、建物、部屋、物理的空間に関する広範な情報が含まれています。データはAI駆動のCortexエンジンを使用します。Matterportデジタルツイン自体に加えて、私たちの空間データは、精密な建物の幾何学と構造的詳細、建物の内容、設備、状態、さらに空間内の多くの視点からの高解像度画像と写真のようにリアルな詳細で構成されています。Cortexは、私たちが深層空間インデックス作成と呼ぶ技術を採用しています。深層空間インデックス作成は、人工知能、コンピュータビジョン、ディープラーニングを使用して、各空間、その構造、およびその内容に関する重要な詳細を精密かつ忠実に識別し伝達します。私たちは、Matterport加入者が任意の建物の相互運用可能なデジタルシステムを活用できる堅牢な空間データ標準を作成しました。
デジタルツインの構築を通じて加入者に高度にインタラクティブなデジタル体験を提供することに加えて、私たちは各加入者に対して2つの質問をします：（1）彼らの建物や物理的空間について重要なことは何か、（2）この空間についてどのような学びと洞察を提供できるか？私たちのAI駆動Cortexエンジンは、空間データライブラリを使用してこれらの質問に答え、集約された不動産トレンドと運用および評価の洞察を提供するのに役立ちます。さらに、Matterportプラットフォームエコシステムが拡大し続けるにつれて、私たちの加入者、パートナー、その他のサードパーティ開発者は、豊富な空間データレイヤーから収穫できる洞察の幅と深さをさらに広げるために独自のツールを持ち込むことができます。
拡張可能なプラットフォームエコシステム
Matterportは、2022年12月31日時点で約920万の空間を管理し、約280億平方フィートをキャプチャした、世界最大かつ最も正確な空間データライブラリを提供しています。私たちの空間データプラットフォームの多用途性と広範なエンタープライズソフトウェア開発キットおよびアプリケーションプログラミングインターフェース（API）により、地理と垂直市場によってMatterportの価値提案を拡張するチャネルとパートナーの堅牢なグローバルエコシステムを開発することができました。私たちは、ターゲット市場全体で統合されたMatterportソリューションを促進するために、パートナーとその加入者との広範なワークフロー統合を継続的に展開する予定です。また、簡単に展開およびアクセスできるMatterportソフトウェアアドオンで空間データプラットフォームの力を拡張するサードパーティソフトウェアマーケットプレイスも開発しています。このマーケットプレイスにより、開発者は新しいアプリケーションと空間データマイニングツールを構築し、Matterport 3Dエクスペリエンスを強化し、私たちのコア製品を補完する新しい生産性と不動産管理ツールを作成できます。サードパーティ開発者によって作成されたこれらの付加価値機能は、スケーラブルな新しい収益ストリームを可能にし、オンラインマーケットプレイスを通じて加入者に展開される各アドオンのサブスクリプションとサービス収益をMatterportが共有します。私たちのプラットフォームエコシステムのネットワーク効果は、ビジネスの成長に貢献し、加入者の粘着性とユーザーエンゲージメントを高めることで、将来の成長をさらに後押しすると信じています。
Matterportのアドオンと拡張機能の例には以下が含まれます：
•アドオン：Encircle（より迅速な請求処理のための使いやすいフィールドドキュメンテーションツール）、WP Matterport Shortcode（Matterportショートコードを使用してMatterportを迅速かつ簡単に埋め込むことができる無料のWordpressプラグイン）、WP3D Models（WordPress + Matterport統合プラグイン）、Rela（リスティング用のオールインワンマーケティングソリューション）、CAPTUR3D（Matterportデジタルツインに価値を拡張するオールインワンコンテンツ管理システム）、Private Model Embed（企業が追加のユーザーライセンスを必要とせずに、企業ネットワーク上の大規模な従業員グループとデジタルツインを非公開で共有できる機能）、Views（グループや大規模組織が異なるタスクを管理するための別個の権限ベースのワークフローを作成できる新しいワークグループコラボレーションフレームワーク）、Guided Tours and Tags（訪問者の関心に合わせて商業または住宅空間の誘導型バーチャルツアーを作成することで訪問者体験を向上させるツール）。Amazon Web Services（AWS）IoT TwinMakerとのプライベートベータ統合を公開し、エンタープライズ顧客がIoTデータを視覚的に没入型で寸法的に正確なMatterportデジタルツインにシームレスに接続できるようにしました。
•サービス：Matterport ADA Compliant Digital Twin（アメリカ障害者法に準拠したデジタルツインを提供するソリューション）およびEnterprise Cloud Software Platform（あらゆるサイズや形状の建物や空間のデジタルツインを作成、公開、管理する、エンタープライズ向けに再考されたクラウドソフトウェアプラットフォーム、屋内外を問わず）。
我々の競争力
私たちは、市場リーダーシップの成長を可能にする多くの競争力を持っていると信じています。私たちの競争力には以下が含まれます：
•Matterportプラットフォームの幅広さと深さ。私たちの中核的な強みは、カスタマイズなしでキャプチャから処理、産業まで多様な垂直市場と地理にわたる広範な到達範囲を持つオールインワンの空間データプラットフォームです。様々なエンタープライズシステムとシームレスに統合する能力により、私たちのプラットフォームは不動産、AEC、旅行とホスピタリティ、修理と保険、産業と施設を含む多様なエンドマーケットの不動産ライフサイクル全体にわたって価値を提供します。2022年12月31日時点で、私たちのグローバルリーチはFortune 1000企業の24％以上を含む170か国以上の加入者に及んでいます。
•市場リーダーシップと先行者優位性。Matterportは約10年前に建築世界のデジタル化とデータ化のカテゴリーを定義し、このカテゴリーでグローバルリーダーになりました。2022年12月31日時点で、私たちのプラットフォーム上に701,000以上の加入者がおり、約920万の空間を管理しています。私たちのリーダーシップは、主にデジタルツイン作成の先行者であったという事実に起因します。先行者優位性の結果として、私たちは継続的に複利効果を生み出し、リーダーシップの地位を強化する深くて豊かな空間データライブラリを蓄積しました。
•重要なネットワーク効果。プラットフォームに新しいキャプチャとデータが追加されるたびに、私たちのデータセットの豊かさと管理下の空間からの洞察の深さが増します。さらに、データを洞察に変換する能力と加入者による新しいデータキャプチャからの増分データを組み合わせることで、Matterportはプラットフォームの加入者向けの機能を開発することができます。私たちは建築世界の空間データライブラリを構築する先行者であり、データの収集と洞察の導出におけるリーダーシップは継続的に複利効果を生み出し、それらの洞察の関連性がさらに新しい加入者を引き付けます。
•加入者に価値ある不動産洞察を提供する生の素材としての巨大な空間データライブラリ。私たちの空間データライブラリの規模は、加入者に洞察を導出する上で重要な利点です。私たちの空間データライブラリは、Cortexにとって重要な基準点として機能し、Matterportが低コストのデジタルカメラやスマートフォンを含む幅広いカメラ技術を使用して強力な3Dデジタルツインを作成することを可能にします。2022年12月31日時点で、私たちのデータは約920万の管理下の空間と約280億平方フィートのキャプチャから得られています。その結果、私たちは不動産の洞察と分析を新しいレベルに引き上げ、様々な産業の加入者に利益をもたらしています。例えば、施設管理者は建物のレイアウトを作成するのに必要な時間を大幅に削減し、現場調査と建設済みモデリングのコストを大幅に削減することができます。AEC加入者は、各建設済み空間の分析を使用してドキュメンテーションを効率化し、容易に協力することができます。
•グローバルリーチとスケール。私たちは、AI駆動の空間データプラットフォームを世界中で継続的に拡大することに焦点を当てています。北米、ヨーロッパ、アジアに大きなプレゼンスを持ち、各地域にリーダーシップチームとゴートゥマーケットインフラストラクチャを有しています。ロンドン、シンガポール、そしてアメリカ合衆国全土にオフィスを構え、国際的な拡大を加速しています。2022年12月31日時点で、170か国以上に701,000以上の加入者がいます。私たちの空間データプラットフォームの地理に依存しない性質は、国際的に成長を続ける上で重要な利点であると信じています。
•10年間のR&Dとイノベーションを支える幅広い特許ポートフォリオ。2022年12月31日時点で、54件の発行済み特許と37件の特許出願中があります。私たちの成功は、約10年間のイノベーションへの集中に基づいています。イノベーションはMatterportの中心にあり、私たちは市場をリードする地位をさらに強化するためにR&Dへの投資を継続的に優先していきます。
•優れたキャプチャ技術。Matterportのキャプチャ技術プラットフォームは、建物や空間のMatterportデジタルツインを作成するために必要な幅広いキャプチャデバイスをサポートするソフトウェアフレームワークです。
これには、LiDARカメラ、360度カメラ、スマートフォン、Matterport Axis、Matterport Pro2およびPro3カメラのサポートが含まれます。Pro2カメラは私たちの空間データ優位性の基礎となり、Matterport対応のサードパーティキャプチャデバイスの配列でその優位性を拡大しました。2022年8月、業界をリードするデジタルツインクラウドプラットフォームの主要な更新とともに、Pro3カメラの出荷を開始しました。Matterport Pro3カメラは高度な3Dキャプチャデバイスで、起動時間の短縮、交換可能なバッテリー、軽量設計を特徴としています。Pro3カメラは屋内外で使用でき、速度、忠実度、多用途性、精度のために設計されています。Pro2カメラとともに、Pro3カメラの今後の販売が私たちのソリューションのさらなる採用を促進すると期待しています。Matterportは3Dキャプチャ体験を民主化し、あらゆる建物タイプと不動産ライフサイクルにおける加入者のニーズに対して、高忠実度で高精度の3Dデジタルツインを容易に利用できるようにしています。他の3Dキャプチャソリューションプロバイダーもありますが、真に寸法的に正確な3D結果を生成できるものはごくわずかで、さらに少数しか自動的に写真のようにリアルな3Dの最終製品をグローバルスケールで作成できません。この広範なキャプチャ技術の提供は、スマートフォンや360度カメラでキャプチャした写真から正確なデジタルツインを自動生成するAI駆動のCortexエンジンを訓練するための豊富な空間データライブラリなしでは不可能でしょう。
</document>"""

# Claudeの回答を出力
print(get_completion(PROMPT))

In [ ]:
# プロンプト
PROMPT = """<question>2020年5月31日の正確な日付におけるMatterportの加入者数は何人でしたか？</question>
以下の文書を読んでください。次に、<scratchpad>タグ内で、文書から最も関連性の高い引用を抽出し、それがユーザーの質問に答えているか、または十分な詳細が不足しているかを検討してください。その後、<answer>タグ内に簡潔な数値回答を書いてください。

<document>
Matterport SEC filing 10-K 2023
Item 1. Business
Our Company
Matterportは、建築世界のデジタル化とデータ化をリードしています。
私たちは、建築世界のデジタル変革が、人々と建物や物理的空間との相互作用の方法を根本的に変えると信じています。
2011年の創業以来、Matterportの先駆的な技術は、建物、空間、場所のオンラインでのデジタル化、アクセス、管理の基準を設定してきました。私たちのプラットフォームの革新的なソフトウェア、空間データ駆動型データサイエンス、3Dキャプチャ技術は、世界最大の資産クラスである建物や物理的空間を長年オフラインで活用不足にしていた障壁を打ち破りました。私たちは、建築世界のデジタル化とデータ化が、大きな運用効率と不動産価値の向上をもたらし続けると信じており、Matterportはこの巨大なグローバル変革をリードするプラットフォームであると考えています。
世界は急速にオフラインからオンラインへ移行しています。
デジタル変革は、今日のあらゆるビジネスと産業に強力で持続的な影響を与えています。International Data Corporation（IDC）によると、2020年から2023年にかけてデジタル変革に6.8兆ドル以上の直接投資が行われ、グローバルデジタル変革支出は2026年に3.4兆ドルに達すると予測されており、5年間の複合年間成長率（CAGR）は16.3％、デジタルツイン投資は5年間のCAGRが35.2％と予想されています。この世俗的なシフトに伴い、建築世界を物理的からデジタルへ移行させる需要も高まっています。それにもかかわらず、建物や空間の大多数はオフラインのままで、デジタル化されていません。Savillsによると、2021年時点で全不動産価値は327兆ドルと推定されていますが、今日でもその大部分はオフラインのままであり、デジタル変革による浸透率は0.1％未満と推定されています。
Matterportは、建築世界のデジタル化の必要性の増大と、建物や空間の理解を促進する空間データ（建物や空間の根底にある固有の詳細）の力を最初に認識した企業の1つです。過去には、技術が物理的な道路地図を、今日私たちが頼りにしているデータ豊富なデジタル地図や位置情報サービスへと進化させました。Matterportは現在、建物をデジタル化し、データ豊富な環境を作り出すことで、私たちが捉える各空間の理解と可能性を大幅に向上させています。ボタン一つで瞬時に都市間の最速ルートを学んだり、最寄りのコーヒーショップを見つけたりできるように、Matterportの建物に関する空間データは、世界中の不動産や空間に関する豊富な洞察と学びを解き放ちます。さらに、今日の地理空間マッピングプラットフォームがマッピングデータを産業界に公開し、ライドシェアリング、電子商取引、フードデリバリーマーケットプレイス、さらには短期レンタルやホームシェアリングなどの新しいビジネスモデルを生み出したように、Matterportの構造化された空間データへのオープンアクセスは、ホスピタリティ、施設管理、保険、建設、不動産、小売などの分野で新たな機会とビジネスモデルを可能にしています。
私たちは、建築世界のデジタル化の総アドレス可能市場機会が2400億ドル以上であり、市場が成熟すると1兆ドルにも達する可能性があると考えています。
これは、世界に現在存在する40億以上の不動産と200億の空間という世界の建物ストックに関する私たちの分析、モデリング、理解に基づいています。人工知能（AI）、機械学習（ML）、ディープラーニング（DL）技術の助けを借りて、強力な空間データ駆動型の不動産洞察と分析からの追加の収益化機会により、建築世界のデジタル化とデータ化の総アドレス可能市場は1兆ドル以上に達すると信じています。
私たちの空間データプラットフォームとデジタルツインの捕捉は、多様な産業とユースケースにわたって価値を提供します。大規模小売業者は数千の店舗の場所をリモートで管理でき、不動産業者は数百の物件と何千人もの訪問者に同時にバーチャルオープンハウスを提供でき、不動産開発業者は建設プロセス全体をより詳細かつ迅速に監視でき、保険会社はより正確に請求と引受評価を文書化し、効率的かつ精密に評価できます。Matterportは、事実上あらゆるサイズ、形状、場所の不動産に関する重要なデジタル体験、ツール、情報を加入者に提供します。
約10年間にわたり、私たちは空間データプラットフォームを成長させ、その機能を拡張して、最も詳細で正確、データが豊富なデジタルツインを作成してきました。さらに、私たちの3D再構成プロセスは完全に自動化されており、世界中のあらゆるタイプ、形状、サイズの何百万もの建物や空間に同等の精度でスケールアップできるソリューションを提供しています。私たちのサービスの普遍的な適用可能性は、多様な垂直市場や地理的領域にわたってMatterportに重要なスケールとリーチを提供しています。2022年12月31日時点で、私たちの加入者ベースは前年同期の503,000人から約39％増加して701,000人以上となり、デジタルツインは170か国以上に到達しています。私たちは複数の産業にわたって280億平方フィート以上の空間をデジタル化しており、これは市場の残りの部分に対して重要なスケールと成長を示しています。

私たちは世界中の建物をデータに変換し続けるにつれて、空間データプラットフォームを拡張して、産業全体の加入者の不動産計画、開発、管理、インテリジェンスをさらに変革し、建築世界のデファクトの建物およびビジネスインテリジェンスエンジンになることを目指しています。私たちは、不動産、建築、エンジニアリング、建設（AEC）、小売、保険、政府を含む産業全体の企業、企業、機関にとって、空間データとそれに基づく洞察への需要が急速に成長し続けると信じています。
私たちは、デジタル化とデータ化がこの巨大なカテゴリーと資産クラス全体で成長のための大きな未開拓の機会を表していると信じています。設計と開発の初期段階からマーケティング、運営、保険、建物の修理とメンテナンスに至るまで、私たちのプラットフォームのソフトウェアと技術は、加入者にコスト削減、収益増加、建物や空間の最適な管理のための重要なツールと洞察を提供します。私たちは、数千億ドルの未実現の利用と運用効率が、私たちの空間データプラットフォームの力を通じて建築世界で解放できると信じています。私たちのプラットフォームとデータソリューションは産業と建物カテゴリー全体で普遍的に適用可能であり、Matterportに大きな利点を与えています。私たちはこの大きな市場機会全体に対応し、世界最大の資産クラスであると信じているものの価値を高めることができます。
加入者に価値を提供してきた実績があり、私たちの提供物にはソフトウェアサブスクリプション、データライセンシング、サービス、製品ハードウェアが含まれます。2022年12月31日時点で、私たちの加入者ベースにはFortune 1000企業の24％以上が含まれており、上位10社の加入者から生成される総収益は10％未満です。2025年までに、収益の80％以上がソフトウェアサブスクリプションとデータライセンスソリューションから来ると予想しています。私たちの革新的な3Dキャプチャ製品であるPro2カメラとPro3カメラは、3D建物および不動産視覚化エコシステムの形成に不可欠な役割を果たしてきました。Pro2カメラとPro3カメラは、私たちのソリューションの採用を促進し、独自の高品質でスケールされたデータセットを生成し、これによって私たちの独自のAIソフトウェアエンジンであるCortexがデジタルツイン作成のパイオニアエンジンになることを可能にしました。Pro2カメラによって最初に促進されたこのデータ優位性により、私たちはキャプチャデバイスに依存しないプラットフォームを開発し、産業や地理にまたがる加入者のために新しい建物と不動産の洞察を生成できるようになりました。
私たちは最近、急速な成長を経験しています。加入者は2018年12月31日から2022年12月31日までの期間に約49倍に成長しました。2022年12月31日に終了した年度の収益は、2021年12月31日に終了した年度の約1億1120万ドルから約22％増加して1億3610万ドルになりました。2022年12月31日に終了した年度の粗利益は、2021年12月31日に終了した年度の6000万ドルから810万ドルまたは14％減少して5180万ドルになりました。これは主に2022年度のグローバルサプライチェーンの制約による特定の混乱的および増分的コストに起因します。既存の加入者が生成するサブスクリプション収益を維持し成長させる能力は、私たちのビジネスの健全性と将来の成長見通しの重要な指標です。私たちは、comparable期間にわたる同じ顧客セットからのネットドル拡大率を測定することで、この分野でのパフォーマンスを追跡しています。2022年12月31日に終了した3か月間のネットドル拡大率103％は、私たちのプラットフォームの粘着性と成長可能性を示しています。
我々の産業と市場機会
今日、建物や空間の大多数はデジタル化されていません。私たちは、現在のサービス可能なアドレス可能市場には約13億の空間が含まれていると推定しており、主に不動産および旅行・ホスピタリティセクターからのものです。2022年12月31日時点で約920万の空間を管理しており、住宅および商業不動産、施設管理、小売、AEC、保険および修理、旅行およびホスピタリティを含む様々なエンドマーケットにわたって世界の建物ストックへの浸透と足跡の拡大を続けています。私たちは、総アドレス可能市場が40億以上の建物と200億の空間を世界的に含み、2400億ドル以上の市場機会をもたらすと推定しています。Matterportの独自の空間データライブラリと不動産データサービスが成長し続けるにつれて、この機会は建物ストックの規模と建物で利用可能な未開拓の価値創造に基づいて1兆ドル以上に増加する可能性があると信じています。COVID-19パンデミックによって作り出された制約は、私たちが過去10年間にわたって多様な産業と市場のために開発してきたスケールされた3Dキャプチャソリューションの重要性をさらに強化し加速させました。

我々の空間データプラットフォーム
概要
私たちの技術プラットフォームは、広範なデジタルキャプチャデバイスから収集された空間データを使用して、物理的な建物や空間を寸法的に正確で写真のようにリアルなデジタルツインに変換し、加入者が以前は利用できなかった建物情報と洞察にアクセスできるようにします。
この巨大な市場で約10年間にわたる最初の参入者として、私たちは業界をリードする3D再構成技術を開発およびスケールアップしてきました。これは、Cortexと呼ばれる私たちの独自のAI駆動ソフトウェアエンジンによって強化されており、機械学習を使用して建物構造全体の写真のようにリアルな3D仮想表現を再現します。これには、コンテンツ、設備、家具が含まれます。完成品は、物理的空間の詳細かつダイナミックな複製であり、Webブラウザからスマートフォンを含むあらゆるデバイスで探索、分析、カスタマイズできます。大規模な商業ビルでさえも管理する力が各加入者の手のひらにあり、これは私たちの高度な技術と空間データ技術スタック全体にわたる画期的なイノベーションによって可能になりました。
私たちの空間データプラットフォームの主要要素には以下が含まれます：
•オフラインの建物をオンラインに。従来、顧客は建物や空間を理解し評価するために現地訪問を行う必要がありました。写真や間取り図は役立つ場合がありますが、これらの二次元（2D）表現形式には限られた情報しかなく、静的で硬直的な傾向があり、そのため各建物や空間の全体的な理解に不可欠な対話的要素が欠けています。私たちの独自のAIソフトウェアであるCortexのAI駆動機能により、物理的オブジェクトの表現はもはや静的な2D画像に限定されず、物理的な訪問を排除できます。Cortexは建物や空間をオフラインからオンラインに移行させ、顧客がリアルタイムでどこからでもオンデマンドでアクセスできるようにします。加入者が建物をスキャンした後、私たちの視覚化アルゴリズムはスキャンを通じてキャプチャされた平坦な2D画像から空間位置と奥行きを正確に推論し、それらを高忠実度で精密なデジタルツインモデルに変換します。これにより、各デジタルツインがプロフェッショナルグレードの画質であることを保証する完全に自動化された画像処理パイプラインが作成されます。
•空間データによって駆動。私たちはデータ駆動型の企業です。空間の各増分キャプチャにより、私たちの空間データライブラリの豊かさと深さが増します。空間データは、人工環境の建物や空間の根底にある固有で特異な詳細を表します。Cortexは、長年にわたって蓄積してきた数十億のデータポイントの幅広さを使用して、デジタルツインの3D精度を向上させます。私たちは加入者がデジタルツイン内のオブジェクトの高さ、位置、その他の特性を正確に特定するのを支援します。私たちの洗練されたアルゴリズムは、データに基づく洞察を生成することで加入者に重要な商業的価値も提供し、不動産に関する評価や決定を自信を持って行うことができます。例えば、不動産開発業者は特定の窓から入る自然な熱と日光の量を評価でき、小売業者は各店舗のレイアウトが同じレベルのコードとブランド要件を満たしていることを確認でき、工場は機械のレイアウトが仕様と場所のガイドラインを満たしていることを確認できます。2022年12月31日時点で約920万の空間を管理しており、私たちの空間データライブラリは建築世界に関する情報のクリアリングハウスとなっています。
•AIとMLによって強化。人工知能と機械学習技術は、空間データを効果的に活用して、動的でリアルで対話的で情報豊富で複数の視点からの表示を可能にする堅牢な仮想体験を作成します。AIとMLはまた、毎日のスキャンに高価なカメラを不要にします—加入者は今やスマートフォンのボタンを押すだけで空間をスキャンできます。その結果、Matterportはデバイスに依存しないプラットフォームとなり、建築世界のデジタル化とインデックス化というミッションに向けてより迅速にスケールアップし推進することができます。
私たちの加入者への価値提案は、設計と建設から保守と運用、販促、販売、リース、保険、修理、修復、セキュリティ、金融に至るまで、デジタル建物ライフサイクル全体をカバーするように設計されています。その結果、私たちは加入者の不動産ポートフォリオ全体をオフラインからオンラインに移行させることで短期的および長期的な投資収益率を向上させる機会を発見するのを支援しながら、加入者とともに収益を成長させるユニークな立場にあると信じています。
ユビキタスキャプチャ
Matterportは3D空間キャプチャの標準となりました。私たちの技術プラットフォームは、世界中の加入者が建物や空間のインタラクティブで寸法的に正確なデジタルツインを迅速、簡単、正確にデジタル化、カスタマイズ、管理できるようにします。
Matterportプラットフォームは、LiDAR、球面、3D、360度カメラ、およびスマートフォンなど、幅広いキャプチャデバイスと連携するように設計されており、すべての加入者のキャプチャニーズに対応します。これにより、あらゆるサイズ、スケール、複雑さの空間を、いつでもどこでもキャプチャする柔軟性が提供されます。
•Matterport Pro3は、以前のバージョンよりも高速に不動産をスキャンし、プロジェクトの完了を加速させる最新の3Dカメラです。Pro3は屋内外の空間の最高精度スキャンを提供し、速度、忠実度、多用途性、精度のために設計されています。1回のスイープで20秒未満、100メートル先まで3Dデータをキャプチャし、Pro3の超高速、高精度LiDARセンサーは何時間も動作し、あらゆる条件で何百万もの測定を行います。
•Matterport Pro2は、世界中で何百万もの空間を高い忠実度、精度、速度、簡単さでキャプチャするために使用されてきた私たちの独自の3Dカメラです。500,000平方フィート以上のサイズの建物をキャプチャできる能力により、多くの住宅、商業、工業、大規模不動産のカメラの選択肢となっています。
•360度カメラ。Matterportは市場で入手可能な360度カメラの選択をサポートしています。これらの手頃な価格のポケットサイズのデバイスは、高忠実度の精密なキャプチャを提供し、小さな家、コンドミニアム、短期賃貸、アパートなどのキャプチャに適しています。これらのデバイスの球面レンズ画像キャプチャ技術は、Cortexにパノラマを業界をリードするデジタルツインに変換するための堅牢で詳細な画像データを提供します。
•LEICA BLK360。Leicaとのパートナーシップを通じて、私たちの3D再構成技術とAI駆動ソフトウェアエンジンであるCortexは、この強力なLiDARカメラをMatterportデジタルツインを作成するための超精密キャプチャデバイスに変換します。これは、正確な精度が必要なAECプロフェッショナルにとって選択されるソリューションです。
•スマートフォンキャプチャ。私たちのキャプチャアプリはiOSとAndroid用に市販されています。Matterportのスマートフォンキャプチャソリューションは3Dキャプチャを民主化し、2020年5月のMatterport for iPhoneの初期導入以来、最新のiPhoneデバイスで誰でも簡単にアクセスして建物や空間を3Dでデジタル化できるようになりました。2021年4月には、Android Captureアプリの正式リリースを発表し、Androidユーザーが迅速かつ簡単に建物や空間を没入型3Dでキャプチャできるようになりました。2022年2月には、Matterport Axisを発売しました。これはスマートフォンを保持するモーター駆動マウントで、Matterport Captureアプリと一緒に使用して、速度、精度、一貫性を向上させながら、あらゆる物理的空間の3Dデジタルツインをキャプチャできます。
Cortexと3D再構成（Matterportデジタルツイン）
2022年12月31日時点で、約920万の空間を管理し、約280億平方フィートの空間をキャプチャした空間データライブラリを持ち、私たちは高度なMLとDL技術を使用して、キャプチャした空間データをアルゴリズム的に任意の物理的空間の正確な3Dデジタル再現に変換します。この知的で自動化された3D再構成は、Cortexによって可能になります。Cortexは私たちのAI駆動ソフトウェアエンジンで、ディープラーニングニューラルネットワークを含み、空間データライブラリを使用して建物や空間がどのように階と部屋に分割されているか、ドアや開口部がどこにあるか、どのタイプの部屋が存在するかを理解し、それらの形式を寸法的な正確さでダイナミックで写真のようにリアルなデジタルツインにコンパイルおよび整列します。Cortexの他のコンポーネントには、建物や空間内のコンテンツを識別し分類するためのAI駆動コンピュータビジョン技術や、家具や設備からドア、窓、照明器具、消火スプリンクラー、非常階段まですべてを識別しセグメント化するオブジェクト認識技術が含まれます。私たちの高度にスケーラブルな人工知能プラットフォームにより、加入者はボタンをクリックするだけで強力な拡張された建物データと洞察にアクセスできます。

Matterportデジタルツインの背後にある科学：Cortex AIのハイライト
MatterportはCortexで動作
Cortexは、任意の建物や空間のデジタルツインを作成するための精密なディープラーニングニューラルネットワークを含むAI駆動ソフトウェアエンジンです。Pro2カメラとPro3カメラでキャプチャした独自の空間データを使用して開発されたCortexは、高度な精度と正確性を提供しながら、日常のデバイスを使用した3Dキャプチャを可能にします。
一般的なニューラルネットワークは現実世界の3D再構成に苦戦します。Matterport最適化ネットワークは、より正確で堅牢な結果を提供します。単なる生のトレーニングデータ以上に、Matterportのデータセットにより、新しいニューラルネットワークアーキテクチャを開発し、何百万もの状況でユーザーの行動と実世界のデータに対して評価することができます。
•ディープラーニング：各空間の詳細なニューラルネットワークデータアーキテクチャを接続し最適化することは、堅牢で高精度の3Dデジタルツインを作成する鍵です。Cortexは、何百万もの建物や空間から集約されたMatterportの豊富な空間データと、世界中の何万人もの加入者によって提供されたそれらのデータの人間による注釈に対して、各3Dモデルを評価し最適化します。Cortexの評価能力と3D再構成のデータ駆動型最適化により、幅広い建物構成、空間、環境にわたって一貫した高精度の結果が得られます。
•ダイナミック3D再構成：2Dビジュアルと静的画像からスケールで精密な3D空間データを作成するには、複数の視点からの写真のようにリアルで詳細なデータと、Cortexのニューラルネットワークと学習能力を訓練および最適化する何百万もの空間の組み合わせが必要です。これにより、任意の空間の3D再構成が向上します。Cortexの機能と私たちの3Dキャプチャ技術におけるリアルタイムの空間アラインメントアルゴリズムを組み合わせることで、進行中の作業の直感的な「プレビュー」が作成され、加入者はコンテンツをインタラクティブにリアルタイムで操作できます。
•コンピュータビジョン：Cortexは、デジタルツインの価値を高める強力な機能のスイートを可能にします。これには、部屋や部屋内のオブジェクトの自動測定、3Dからの自動2D高解像度フォトギャラリー作成、プライバシー保護のための自動顔のぼかし、カスタムビデオ、ウォークスルー、自動部屋ラベリング、オブジェクト認識が含まれます。
•高度な画像処理：Matterportの計算写真アルゴリズムは、各デジタルツインがプロフェッショナルグレードの画質であることを保証するために、完全に自動化された画像処理パイプラインを作成します。私たちの特許取得技術により、3Dキャプチャは単一のボタンを押すだけで簡単になります。Matterportのソフトウェアと技術が残りのステップを管理し、ホワイトバランスとカメラ固有の色補正、高ダイナミックレンジのトーンマッピング、ノイズ除去、ヘイズ除去、シャープニング、彩度、その他の画質を向上させる調整を行います。
空間データとAI駆動の洞察
すべてのMatterportデジタルツインには、建物、部屋、物理的空間に関する広範な情報が含まれています。データはAI駆動のCortexエンジンを使用します。Matterportデジタルツイン自体に加えて、私たちの空間データは、精密な建物の幾何学と構造的詳細、建物の内容、設備、状態、さらに空間内の多くの視点からの高解像度画像と写真のようにリアルな詳細で構成されています。Cortexは、私たちが深層空間インデックス作成と呼ぶ技術を採用しています。深層空間インデックス作成は、人工知能、コンピュータビジョン、ディープラーニングを使用して、各空間、その構造、およびその内容に関する重要な詳細を精密かつ忠実に識別し伝達します。私たちは、Matterport加入者が任意の建物の相互運用可能なデジタルシステムを活用できる堅牢な空間データ標準を作成しました。
デジタルツインの構築を通じて加入者に高度にインタラクティブなデジタル体験を提供することに加えて、私たちは各加入者に対して2つの質問をします：（1）彼らの建物や物理的空間について重要なことは何か、（2）この空間についてどのような学びと洞察を提供できるか？私たちのAI駆動Cortexエンジンは、空間データライブラリを使用してこれらの質問に答え、集約された不動産トレンドと運用および評価の洞察を提供するのに役立ちます。さらに、Matterportプラットフォームエコシステムが拡大し続けるにつれて、私たちの加入者、パートナー、その他のサードパーティ開発者は、豊富な空間データレイヤーから収穫できる洞察の幅と深さをさらに広げるために独自のツールを持ち込むことができます。
拡張可能なプラットフォームエコシステム
Matterportは、2022年12月31日時点で約920万の空間を管理し、約280億平方フィートをキャプチャした、世界最大かつ最も正確な空間データライブラリを提供しています。私たちの空間データプラットフォームの多用途性と広範なエンタープライズソフトウェア開発キットおよびアプリケーションプログラミングインターフェース（API）により、地理と垂直市場によってMatterportの価値提案を拡張するチャネルとパートナーの堅牢なグローバルエコシステムを開発することができました。私たちは、ターゲット市場全体で統合されたMatterportソリューションを促進するために、パートナーとその加入者との広範なワークフロー統合を継続的に展開する予定です。また、簡単に展開およびアクセスできるMatterportソフトウェアアドオンで空間データプラットフォームの力を拡張するサードパーティソフトウェアマーケットプレイスも開発しています。このマーケットプレイスにより、開発者は新しいアプリケーションと空間データマイニングツールを構築し、Matterport 3Dエクスペリエンスを強化し、私たちのコア製品を補完する新しい生産性と不動産管理ツールを作成できます。サードパーティ開発者によって作成されたこれらの付加価値機能は、スケーラブルな新しい収益ストリームを可能にし、オンラインマーケットプレイスを通じて加入者に展開される各アドオンのサブスクリプションとサービス収益をMatterportが共有します。私たちのプラットフォームエコシステムのネットワーク効果は、ビジネスの成長に貢献し、加入者の粘着性とユーザーエンゲージメントを高めることで、将来の成長をさらに後押しすると信じています。
Matterportのアドオンと拡張機能の例には以下が含まれます：
•アドオン：Encircle（より迅速な請求処理のための使いやすいフィールドドキュメンテーションツール）、WP Matterport Shortcode（Matterportショートコードを使用してMatterportを迅速かつ簡単に埋め込むことができる無料のWordpressプラグイン）、WP3D Models（WordPress + Matterport統合プラグイン）、Rela（リスティング用のオールインワンマーケティングソリューション）、CAPTUR3D（Matterportデジタルツインに価値を拡張するオールインワンコンテンツ管理システム）、Private Model Embed（企業が追加のユーザーライセンスを必要とせずに、企業ネットワーク上の大規模な従業員グループとデジタルツインを非公開で共有できる機能）、Views（グループや大規模組織が異なるタスクを管理するための別個の権限ベースのワークフローを作成できる新しいワークグループコラボレーションフレームワーク）、Guided Tours and Tags（訪問者の関心に合わせて商業または住宅空間の誘導型バーチャルツアーを作成することで訪問者体験を向上させるツール）。Amazon Web Services（AWS）IoT TwinMakerとのプライベートベータ統合を公開し、エンタープライズ顧客がIoTデータを視覚的に没入型で寸法的に正確なMatterportデジタルツインにシームレスに接続できるようにしました。
•サービス：Matterport ADA Compliant Digital Twin（アメリカ障害者法に準拠したデジタルツインを提供するソリューション）およびEnterprise Cloud Software Platform（あらゆるサイズや形状の建物や空間のデジタルツインを作成、公開、管理する、エンタープライズ向けに再考されたクラウドソフトウェアプラットフォーム、屋内外を問わず）。
我々の競争力
私たちは、市場リーダーシップの成長を可能にする多くの競争力を持っていると信じています。私たちの競争力には以下が含まれます：
•Matterportプラットフォームの幅広さと深さ。私たちの中核的な強みは、カスタマイズなしでキャプチャから処理、産業まで多様な垂直市場と地理にわたる広範な到達範囲を持つオールインワンの空間データプラットフォームです。様々なエンタープライズシステムとシームレスに統合する能力により、私たちのプラットフォームは不動産、AEC、旅行とホスピタリティ、修理と保険、産業と施設を含む多様なエンドマーケットの不動産ライフサイクル全体にわたって価値を提供します。2022年12月31日時点で、私たちのグローバルリーチはFortune 1000企業の24％以上を含む170か国以上の加入者に及んでいます。
•市場リーダーシップと先行者優位性。Matterportは約10年前に建築世界のデジタル化とデータ化のカテゴリーを定義し、このカテゴリーでグローバルリーダーになりました。2022年12月31日時点で、私たちのプラットフォーム上に701,000以上の加入者がおり、約920万の空間を管理しています。私たちのリーダーシップは、主にデジタルツイン作成の先行者であったという事実に起因します。先行者優位性の結果として、私たちは継続的に複利効果を生み出し、リーダーシップの地位を強化する深くて豊かな空間データライブラリを蓄積しました。
•重要なネットワーク効果。プラットフォームに新しいキャプチャとデータが追加されるたびに、私たちのデータセットの豊かさと管理下の空間からの洞察の深さが増します。さらに、データを洞察に変換する能力と加入者による新しいデータキャプチャからの増分データを組み合わせることで、Matterportはプラットフォームの加入者向けの機能を開発することができます。私たちは建築世界の空間データライブラリを構築する先行者であり、データの収集と洞察の導出におけるリーダーシップは継続的に複利効果を生み出し、それらの洞察の関連性がさらに新しい加入者を引き付けます。
•加入者に価値ある不動産洞察を提供する生の素材としての巨大な空間データライブラリ。私たちの空間データライブラリの規模は、加入者に洞察を導出する上で重要な利点です。私たちの空間データライブラリは、Cortexにとって重要な基準点として機能し、Matterportが低コストのデジタルカメラやスマートフォンを含む幅広いカメラ技術を使用して強力な3Dデジタルツインを作成することを可能にします。2022年12月31日時点で、私たちのデータは約920万の管理下の空間と約280億平方フィートのキャプチャから得られています。その結果、私たちは不動産の洞察と分析を新しいレベルに引き上げ、様々な産業の加入者に利益をもたらしています。例えば、施設管理者は建物のレイアウトを作成するのに必要な時間を大幅に削減し、現場調査と建設済みモデリングのコストを大幅に削減することができます。AEC加入者は、各建設済み空間の分析を使用してドキュメンテーションを効率化し、容易に協力することができます。
•グローバルリーチとスケール。私たちは、AI駆動の空間データプラットフォームを世界中で継続的に拡大することに焦点を当てています。北米、ヨーロッパ、アジアに大きなプレゼンスを持ち、各地域にリーダーシップチームとゴートゥマーケットインフラストラクチャを有しています。ロンドン、シンガポール、そしてアメリカ合衆国全土にオフィスを構え、国際的な拡大を加速しています。2022年12月31日時点で、170か国以上に701,000以上の加入者がいます。私たちの空間データプラットフォームの地理に依存しない性質は、国際的に成長を続ける上で重要な利点であると信じています。
•10年間のR&Dとイノベーションを支える幅広い特許ポートフォリオ。2022年12月31日時点で、54件の発行済み特許と37件の特許出願中があります。私たちの成功は、約10年間のイノベーションへの集中に基づいています。イノベーションはMatterportの中心にあり、私たちは市場をリードする地位をさらに強化するためにR&Dへの投資を継続的に優先していきます。
•優れたキャプチャ技術。Matterportのキャプチャ技術プラットフォームは、建物や空間のMatterportデジタルツインを作成するために必要な幅広いキャプチャデバイスをサポートするソフトウェアフレームワークです。
これには、LiDARカメラ、360度カメラ、スマートフォン、Matterport Axis、Matterport Pro2およびPro3カメラのサポートが含まれます。Pro2カメラは私たちの空間データ優位性の基礎となり、Matterport対応のサードパーティキャプチャデバイスの配列でその優位性を拡大しました。2022年8月、業界をリードするデジタルツインクラウドプラットフォームの主要な更新とともに、Pro3カメラの出荷を開始しました。Matterport Pro3カメラは高度な3Dキャプチャデバイスで、起動時間の短縮、交換可能なバッテリー、軽量設計を特徴としています。Pro3カメラは屋内外で使用でき、速度、忠実度、多用途性、精度のために設計されています。Pro2カメラとともに、Pro3カメラの今後の販売が私たちのソリューションのさらなる採用を促進すると期待しています。Matterportは3Dキャプチャ体験を民主化し、あらゆる建物タイプと不動産ライフサイクルにおける加入者のニーズに対して、高忠実度で高精度の3Dデジタルツインを容易に利用できるようにしています。他の3Dキャプチャソリューションプロバイダーもありますが、真に寸法的に正確な3D結果を生成できるものはごくわずかで、さらに少数しか自動的に写真のようにリアルな3Dの最終製品をグローバルスケールで作成できません。この広範なキャプチャ技術の提供は、スマートフォンや360度カメラでキャプチャした写真から正確なデジタルツインを自動生成するAI駆動のCortexエンジンを訓練するための豊富な空間データライブラリなしでは不可能でしょう。
</document>"""

# Claudeの回答を出力
print(get_completion(PROMPT))